In [1]:
from pyomo.environ import *

In [2]:
def solve_pyomo(para):
    p_ei  = para['pei']
    p_er  = para['per']
    p_ea  = 1-p_ei-p_er
    p_ai  = p_ei
    p_ar  = p_er
    p_irr = 1/8
    alpha  = 0.6
    beta_1 = para['beta1']
    beta_2 = para['beta2']
    cost_tr = 0.0977
    p_id  = 1.1/1000
    p_idd = 0.1221/1000
    p_srr = 0.7/3
    S = 100000
    E = 20
    A = 0
    I = 0
    R = 0
    D = 0
    N = S+E+A+I+R+D
    K = 50
    w_max  = 1.0
    tr_max = 1.0
    v_max = 0.2/14
    M = 3.6e3
    
    m = ConcreteModel()
    m.k = RangeSet(0, K)
    m.t = RangeSet(0, K-1)
    m.N = N
    m.tri = Var(m.k, bounds=(0, 1)) 
    m.wi = Var(m.k, bounds=(0, 1))  
    m.v = Var(m.k, bounds=(0, 1))
    m.aq = Var(m.k, bounds=(0, 1))
    m.iq = Var(m.k, bounds=(0, 1))
    m.x_s = Var(m.k, domain = NonNegativeReals)
    m.x_e = Var(m.k, domain = NonNegativeReals)
    m.x_a = Var(m.k, domain = NonNegativeReals)
    m.x_i = Var(m.k, domain = NonNegativeReals)
    m.x_r = Var(m.k, domain = NonNegativeReals)
    m.x_d = Var(m.k, domain = NonNegativeReals)
    
    m.xs_c = Constraint(m.t, rule=lambda m, k: m.x_s[k+1] == m.x_s[k]
                    -(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                    *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                    -p_srr*m.v[k]*v_max*m.x_s[k])

    m.xe_c = Constraint(m.t, rule=lambda m, k: m.x_e[k+1] == m.x_e[k]
                        +alpha*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        -p_ei*m.x_e[k]
                        -p_er*m.x_e[k]
                        -m.wi[k]*p_ea*m.x_e[k])
                     
    m.xa_c = Constraint(m.t, rule=lambda m, k: m.x_a[k+1] == m.x_a[k]
                        +m.wi[k]*p_ea*m.x_e[k]
                        -p_ai*m.x_a[k]
                        -p_ar*m.x_a[k])
                                        
    m.xi_c = Constraint(m.t, rule=lambda m, k: m.x_i[k+1] == m.x_i[k]
                        +(1-alpha)*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +p_ei*m.x_e[k]
                        +p_ai*m.x_a[k]
                        -m.tri[k]*p_irr*m.x_i[k]
                        -(1-m.tri[k])*m.x_i[k]*p_id
                        -m.tri[k]*m.x_i[k]*p_idd)
                                                                                       
    m.xr_c = Constraint(m.t, rule=lambda m, k: m.x_r[k+1] == m.x_r[k]
                        +m.tri[k]*p_irr*m.x_i[k]
                        +p_er*m.x_e[k]
                        +p_ar*m.x_a[k]
                        +p_srr*m.v[k]*v_max*m.x_s[k])

    m.xd_c = Constraint(m.t, rule=lambda m, k: m.x_d[k+1] == m.x_d[k]
                        +(1-m.tri[k])*m.x_i[k]*p_id
                        +m.tri[k]*m.x_i[k]*p_idd)
    
    m.pc = ConstraintList()
    m.pc.add(m.x_s[0]==S)
    m.pc.add(m.x_e[0]==E)
    m.pc.add(m.x_a[0]==A)
    m.pc.add(m.x_i[0]==I)
    m.pc.add(m.x_r[0]==R)
    m.pc.add(m.x_d[0]==D)

    gamma = 1
    m.sumcost = sum(m.wi[k]*(0.000369*(m.wi[k]*(m.x_s[k]+m.x_a[k]+m.x_e[k]+m.x_r[k])/m.N)
                         *(m.wi[k]*(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])/m.N) + 0.001057)
                         *(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])
                    +0.0977*m.tri[k]*m.x_i[k]+m.x_s[k]
                    *m.v[k]*v_max*0.07 + m.x_i[k]*m.iq[k]*0.02 + m.x_a[k]*m.aq[k]*0.02 for k in m.t) 
    m.budget_c = Constraint(expr = m.sumcost <= M)

    m.suminfected = sum(2.6 * (beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +(m.x_d[k+1] - m.x_d[k]) * 60 for k in m.t)
    m.obj = Objective(expr = m.suminfected, sense = minimize)
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 100000
    try:
        solver.solve(m,tee=True)
    except:
        print()

In [3]:
import pickle
interval = 0.3
para_truth = pickle.load(open('para_truth' + str(round(interval * 100)), 'rb'))

In [4]:
solve_pyomo(para_truth[0])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 277r 7.6257295e+04 8.09e+04 9.99e+02   4.9 0.00e+00   1.7 0.00e+00 1.52e-07R  2
 278r 7.6252953e+04 8.09e+04 4.77e+03   4.9 7.34e+06    -  8.97e-05 8.75e-07f  1
 279r 8.1681092e+04 7.94e+04 5.63e+03   2.8 3.26e+05    -  6.51e-03 4.54e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280r 8.3279284e+04 7.90e+04 9.29e+03   2.8 4.24e+05    -  6.95e-03 9.85e-04f  1
 281r 8.6238369e+04 7.82e+04 1.25e+04   2.8 4.43e+05    -  3.52e-03 1.69e-03f  1
 282r 9.2339134e+04 7.67e+04 1.29e+04   2.8 4.61e+05    -  3.71e-03 3.30e-03f  1
 283r 1.0149981e+05 7.44e+04 2.88e+03   2.8 4.86e+05    -  2.64e-03 4.67e-03f  1
 284r 1.0156456e+05 7.44e+04 3.59e+05   2.1 1.16e+01   2.0 9.92e-01 2.57e-01f  1
 285r 1.0140865e+05 7.44e+04 1.02e+03   2.1 3.06e+01   1.5 1.00e+00 1.00e+00f  1
 286r 1.0133753e+05 7.44e+04 2.51e+04   0.7 9.06e+01   1.0 5.86e-01 3.10e-01f  1
 287r 1.0103028e+05 7.44e+04 2.18e+04   0.7 2.85e+02   0.6 1.11e-01 1.05e-01f  1
 288r 1.0093318e+05 7.44e+04

 451  3.9309363e+05 3.99e+03 1.01e+04  -1.0 3.99e+03  -1.9 6.03e-06 9.06e-05h  1
 452  3.9309460e+05 3.99e+03 1.01e+04  -1.0 3.99e+03  -1.5 4.28e-04 3.08e-05h  1
 453  3.9309644e+05 3.99e+03 1.01e+04  -1.0 7.15e+03  -1.9 5.15e-05 6.04e-05h  1
 454  3.9309693e+05 3.99e+03 1.01e+04  -1.0 3.99e+03  -1.5 2.47e-04 1.66e-05h  1
 455  3.9309991e+05 3.99e+03 1.01e+04  -1.0 3.99e+03  -1.1 2.14e-04 9.28e-05h  1
 456  3.9310131e+05 3.99e+03 1.01e+04  -1.0 4.53e+03  -1.6 6.81e-04 5.48e-05h  1
 457  3.9310333e+05 3.99e+03 1.01e+04  -1.0 4.43e+03  -1.1 4.26e-05 7.47e-05h  1
 458  3.9310448e+05 3.99e+03 1.01e+04  -1.0 4.70e+03  -1.6 1.36e-03 4.85e-05h  1
 459  3.9310599e+05 3.99e+03 1.01e+04  -1.0 6.10e+03  -2.1 1.38e-03 7.52e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 460  3.9310673e+05 3.99e+03 1.01e+04  -1.0 5.92e+03  -1.7 3.01e-04 3.96e-05h  1
 461  3.9311207e+05 3.99e+03 1.01e+04  -1.0 6.07e+03  -2.2 2.03e-04 3.19e-04h  1
 462r 3.9311207e+05 3.99e+03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570  3.8992846e+05 3.52e+02 7.54e+04  -1.0 2.46e+04  -5.0 1.96e-02 1.04e-02f  1
 571  3.8811826e+05 3.35e+02 8.14e+04  -1.0 2.39e+04  -4.6 1.33e-03 1.02e-01f  1
 572  3.8763444e+05 3.33e+02 8.07e+04  -1.0 3.33e+04  -5.1 1.44e-02 1.99e-02f  1
 573  3.8761005e+05 3.33e+02 8.05e+04  -1.0 2.29e+04  -4.7 1.69e-03 8.16e-04f  1
 574  3.8760061e+05 3.33e+02 8.04e+04  -1.0 3.40e+05  -5.1 2.90e-04 5.11e-05f  1
 575  3.8747549e+05 3.32e+02 7.62e+04  -1.0 2.29e+04  -4.7 3.11e-02 3.71e-03f  1
 576  3.8744341e+05 3.31e+02 7.62e+04  -1.0 4.88e+04  -5.2 2.14e-04 7.78e-04f  1
 577  3.8734374e+05 3.28e+02 7.56e+04  -1.0 2.08e+04  -3.9 1.76e-02 1.03e-02f  1
 578  3.8731799e+05 3.27e+02 7.19e+04  -1.0 2.25e+04  -4.3 3.94e-02 1.84e-03f  1
 579  3.8618289e+05 3.17e+02 7.54e+04  -1.0 2.51e+04    -  8.72e-04 3.95e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 580  3.8603146e+05 3.15e+02

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 690  3.4853204e+05 3.42e+01 3.49e+03  -1.0 7.64e+03  -4.7 3.64e-01 2.04e-02f  1
 691  3.4815300e+05 3.28e+01 3.28e+03  -1.0 7.54e+03  -5.2 6.02e-02 5.82e-02f  1
 692  3.4674018e+05 3.29e+01 2.43e+03  -1.0 7.22e+03  -4.7 4.77e-01 2.35e-01f  1
 693  3.4674020e+05 3.29e+01 2.40e+03  -1.0 3.87e+04   0.2 9.60e-07 2.27e-06f  1
 694  3.4674024e+05 3.29e+01 1.52e+04  -1.0 2.31e+01   0.6 1.29e-02 5.13e-05h  1
 695  3.4674030e+05 3.29e+01 2.04e+04  -1.0 9.83e+01   0.2 2.44e-03 1.18e-04h  1
 696  3.4674028e+05 3.29e+01 2.58e+04  -1.0 7.35e+02  -0.3 7.46e-04 6.31e-05h  1
 697  3.4674035e+05 3.29e+01 2.56e+04  -1.0 2.44e+03  -0.8 2.02e-04 2.30e-04h  1
 698  3.4674145e+05 3.29e+01 2.22e+04  -1.0 3.39e+03  -1.3 8.57e-05 7.59e-04h  1
 699  3.4674246e+05 3.29e+01 4.47e+04  -1.0 3.69e+03  -1.7 1.25e-02 5.36e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 700  3.4678441e+05 3.22e+01

...

2689  9.7823760e+01 2.99e-01 4.49e+03  -5.7 5.82e+01  -1.7 7.42e-04 2.61e-08h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2690  9.7743760e+01 2.98e-01 4.49e+03  -5.7 2.10e+01  -2.2 7.14e-03 3.61e-03h  1
2691  9.7743757e+01 2.98e-01 4.72e+03  -5.7 5.88e+00  -0.9 1.34e-02 8.53e-08h  2
2692  9.7716621e+01 2.94e-01 4.26e+03  -5.7 2.32e+00  -1.3 7.17e-02 1.28e-02h  1
2693  9.7625203e+01 2.91e-01 3.67e+03  -5.7 6.53e+00  -1.8 2.22e-01 1.24e-02h  1
2694  9.5466710e+01 2.68e-01 3.18e+03  -5.7 2.66e+01  -2.3 4.72e-02 7.84e-02f  1
2695  9.3566391e+01 2.05e-01 2.51e+03  -5.7 7.36e+00  -1.9 7.11e-02 2.34e-01f  1
2696  9.3269000e+01 2.03e-01 2.61e+03  -5.7 2.91e+01  -2.3 2.29e-02 1.00e-02f  1
2697  9.1678707e+01 2.03e-01 2.59e+03  -5.7 1.06e+03  -2.8 1.56e-03 2.19e-03f  1
2698  9.1561568e+01 2.02e-01 2.69e+03  -5.7 3.79e+01  -2.4 1.23e-02 3.26e-03f  1
2699  9.1189938e+01 1.94e-01 4.48e+03  -5.7 9.19e+00  -2.0 1.00e+00 3.86e-02f  1
iter    objective    inf_pr 

2795  6.9466838e+01 3.94e-02 4.76e+02  -5.7 6.28e+02  -6.5 1.23e-01 3.58e-07f  2
2796  6.9448645e+01 4.07e-02 3.63e+02  -5.7 2.07e+03  -7.0 9.96e-02 1.10e-02f  1
2797  6.9415253e+01 5.50e-02 3.54e+02  -5.7 4.10e+03  -7.5 1.57e-02 1.42e-02f  1
2798  6.9411779e+01 5.49e-02 2.41e+02  -5.7 1.08e+03  -7.1 1.04e-01 5.79e-03h  1
2799  6.9383581e+01 6.29e-02 3.47e+02  -5.7 2.52e+03  -7.6 1.00e-01 1.86e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2800  6.9372031e+01 6.27e-02 3.81e+02  -5.7 8.08e+02  -7.1 7.90e-02 2.01e-02h  1
2801  6.9370874e+01 6.27e-02 3.74e+02  -5.7 3.05e+03  -6.7 2.68e-07 3.57e-05h  1
2802  6.9368357e+01 6.27e-02 4.68e+03  -5.7 1.30e+03  -6.3 4.84e-02 1.79e-04f  1
2803  6.9350852e+01 6.41e-02 4.57e+03  -5.7 4.00e+04  -6.8 7.71e-05 4.02e-04f  1
2804  6.8582672e+01 3.88e-01 1.68e+03  -5.7 1.36e+03  -6.3 2.25e-02 5.20e-02f  1
2805  6.8432296e+01 4.14e-01 4.41e+03  -5.7 5.37e+02  -6.8 1.11e-01 2.73e-02f  1
2806  6.8196908e+01 5.09e-01

2902  6.4291030e+01 3.08e-08 6.87e-05  -5.7 1.53e-01  -3.3 1.00e+00 1.00e+00h  1
2903  6.4290887e+01 1.25e-08 9.75e-01  -5.7 4.28e-01  -3.8 1.00e+00 1.71e-01h  1
2904  6.4289757e+01 2.50e-07 1.57e+00  -5.7 1.15e+00  -4.3 1.00e+00 5.05e-01f  1
2905  6.4288520e+01 6.67e-07 7.51e+00  -5.7 3.37e+00  -4.8 1.00e+00 1.90e-01f  1
2906  6.4276590e+01 8.57e-03 9.25e+00  -5.7 9.60e+00  -5.3 1.00e+00 6.48e-01f  1
2907  6.4259605e+01 2.36e-02 4.69e+01  -5.7 2.83e+01  -5.7 1.00e+00 3.41e-01f  1
2908  6.4229756e+01 7.53e-02 5.36e+01  -5.7 7.02e+01  -6.2 3.55e-01 2.29e-01f  1
2909  6.4230197e+01 2.52e-11 3.07e-02  -5.7 7.13e-04   1.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2910  6.4230190e+01 4.24e-11 3.26e-03  -5.7 2.28e-04   1.2 1.00e+00 1.00e+00h  1
2911  6.4230188e+01 1.46e-11 6.14e-04  -5.7 1.28e-04   0.7 1.00e+00 1.00e+00h  1
2912  6.4230188e+01 1.46e-11 3.74e-04  -5.7 2.35e-04   0.2 1.00e+00 1.00e+00h  1
2913  6.4230188e+01 1.46e-11

3008  6.3785246e+01 8.75e-05 7.54e-04  -8.6 6.40e-01  -4.9 1.00e+00 1.00e+00h  1
3009  6.3784116e+01 7.93e-04 6.75e-03  -8.6 1.92e+00  -5.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3010  6.3781233e+01 5.30e-03 2.07e+01  -8.6 5.71e+00  -5.9 7.59e-01 8.59e-01h  1
3011  6.3779975e+01 5.62e-03 1.84e+02  -8.6 1.68e+01  -6.3 4.52e-01 1.29e-01f  1
3012  6.3779975e+01 5.62e-03 1.84e+02  -8.6 4.81e+01  -6.8 6.25e-05 7.86e-06h  1
In iteration 3012, 1 Slack too small, adjusting variable bound
3013  6.3779975e+01 5.62e-03 3.42e+02  -8.6 5.06e+01  -7.3 9.76e-02 1.41e-07h  1
3014  6.3779904e+01 5.38e-03 3.01e+02  -8.6 1.06e+02  -7.8 1.14e-01 5.77e-02h  1
In iteration 3014, 1 Slack too small, adjusting variable bound
3015  6.3779139e+01 1.85e-02 3.00e+02  -8.6 3.27e+02  -8.2 4.99e-02 2.11e-01h  1
3016  6.3778977e+01 1.90e-02 2.41e+02  -8.6 1.07e+03  -8.7 1.12e-01 1.46e-02h  1
3017  6.3778956e+01 1.90e-02 2.18e+02  -8.6 8.07e+03  -9.2 1.86e

3128  6.3663223e+01 2.49e+00 6.58e+01  -8.6 1.85e+02    -  3.32e-01 1.89e-02h  1
3129  6.3654482e+01 2.27e+00 6.19e+01  -8.6 2.23e+02    -  1.09e-01 9.60e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3130  6.3645156e+01 2.03e+00 5.61e+01  -8.6 2.79e+02    -  1.20e-01 1.14e-01h  1
3131  6.3630002e+01 1.50e+00 4.52e+02  -8.6 2.60e-02  -1.6 8.85e-01 2.15e-01h  1
3132  6.3626174e+01 1.35e+00 1.89e+03  -8.6 8.47e-02  -2.1 7.04e-01 7.68e-02h  1
3133  6.3623066e+01 1.22e+00 2.08e+03  -8.6 2.16e-01  -2.6 1.81e-01 7.85e-02h  1
3134  6.3620479e+01 1.09e+00 4.54e+03  -8.6 9.79e-02  -2.1 8.40e-01 7.52e-02h  1
3135  6.3619524e+01 1.05e+00 4.43e+03  -8.6 5.29e-01  -2.6 5.28e-02 3.27e-02h  1
3136  6.3618362e+01 9.90e-01 5.87e+03  -8.6 1.58e-01  -2.2 9.10e-01 4.24e-02h  1
3137  6.3618318e+01 9.89e-01 5.88e+03  -8.6 6.79e+00  -2.7 4.01e-03 1.56e-03h  1
3138  6.3617462e+01 9.41e-01 6.21e+03  -8.6 1.49e-01  -2.2 9.65e-01 3.39e-02h  1
3139  6.3614623e+01 7.78e-01

In [5]:
solve_pyomo(para_truth[1])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 7.9839808e+03 9.79e+04 4.97e+02  -3.0 6.57e+01  -0.9 5.48e-01 2.98e-01f  1
 188r 8.0491418e+03 9.79e+04 2.23e+02  -3.0 2.56e+01  -0.5 1.00e+00 6.16e-01f  1
 189r 8.1335492e+03 9.79e+04 8.93e+02  -3.0 6.86e+01  -1.0 8.20e-01 2.90e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 8.2247208e+03 9.79e+04 1.19e+03  -3.0 1.62e+02  -1.4 2.61e-01 1.26e-01f  1
 191r 8.2366489e+03 9.79e+04 2.29e+03  -3.0 6.68e+02    -  7.96e-02 3.56e-03f  1
 192r 8.3320992e+03 9.78e+04 1.16e+03  -3.0 7.07e+01  -1.0 1.00e+00 3.12e-01f  1
 193r 8.3514155e+03 9.78e+04 1.65e+03  -3.0 6.44e+02    -  4.11e-02 5.96e-03f  1
 194r 8.4692701e+03 9.78e+04 2.63e+01  -3.0 2.75e+01  -0.6 9.73e-01 1.00e+00f  1
 195r 8.5480098e+03 9.78e+04 6.11e+02  -3.0 7.08e+01  -1.1 5.97e-01 2.50e-01f  1
 196r 8.6320431e+03 9.78e+04 2.16e+02  -3.0 2.81e+01  -0.6 1.00e+00 6.83e-01f  1
 197r 8.7539438e+03 9.77e+04 1.01e+03  -3.0 7.26e+01  -1.1 9.23e-01 3.71e-01f  1
 198r 8.8267423e+03 9.77e+04

 479r 7.8431076e+04 8.08e+04 9.99e+02   4.9 0.00e+00   3.0 0.00e+00 2.48e-09R  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480r 7.8428353e+04 8.08e+04 9.99e+02   4.9 9.77e+08    -  8.47e-09 4.20e-09f  1
 481r 7.8424106e+04 8.08e+04 9.99e+02   2.8 1.81e+06    -  1.07e-05 8.76e-06f  1
 482r 7.8522915e+04 8.08e+04 9.99e+02   2.8 9.50e+05    -  1.61e-04 4.06e-05f  1
 483r 7.8975431e+04 8.07e+04 3.74e+03   2.8 6.16e+05    -  3.35e-03 1.76e-04f  1
 484r 8.3755123e+04 7.95e+04 6.86e+03   2.8 7.63e+05    -  6.17e-03 1.52e-03f  1
 485r 9.2097283e+04 7.75e+04 5.85e+03   2.8 1.11e+06    -  3.84e-04 1.79e-03f  1
 486r 9.4032205e+04 7.71e+04 1.01e+04   2.8 1.22e+06    -  5.79e-03 3.84e-04f  1
 487r 1.0097756e+05 7.54e+04 9.07e+03   2.8 1.65e+06    -  3.04e-04 1.03e-03f  1
 488r 1.0433965e+05 7.46e+04 9.13e+03   2.8 1.76e+06    -  5.15e-04 4.64e-04f  1
 489r 1.0760424e+05 7.38e+04 1.03e+04   2.8 1.83e+06    -  8.44e-04 4.24e-04f  1
iter    objective    inf_pr 

 607  4.2301284e+05 6.69e+02 4.71e+05  -1.0 2.61e+04  -2.7 4.19e-04 1.40e-04f  1
 608  4.2299914e+05 6.68e+02 4.68e+05  -1.0 2.67e+04  -3.1 7.94e-06 1.68e-03f  1
 609  4.2299880e+05 6.68e+02 5.71e+05  -1.0 2.62e+04  -2.7 2.52e-02 4.96e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  4.2296402e+05 6.65e+02 5.68e+05  -1.0 2.69e+04  -3.2 3.72e-03 4.22e-03f  1
 611  4.2296236e+05 6.65e+02 7.41e+05  -1.0 2.70e+04  -2.8 4.87e-02 2.33e-04f  1
 612  4.2290255e+05 6.60e+02 7.35e+05  -1.0 2.73e+04  -3.2 7.18e-03 7.19e-03f  1
 613  4.2290013e+05 6.60e+02 7.34e+05  -1.0 2.76e+04  -2.8 7.46e-05 3.37e-04f  1
 614  4.2288835e+05 6.59e+02 7.53e+05  -1.0 2.76e+04  -3.3 4.10e-02 1.35e-03f  1
 615  4.2286475e+05 6.57e+02 7.55e+05  -1.0 2.76e+04  -2.9 5.01e-03 3.24e-03f  1
 616  4.2285790e+05 6.56e+02 7.56e+05  -1.0 2.72e+04  -3.3 5.11e-03 7.28e-04f  1
 617  4.2283893e+05 6.55e+02 8.53e+05  -1.0 2.72e+04  -2.9 4.72e-02 2.32e-03f  1
 618  4.2275139e+05 6.49e+02

 732  3.4533808e+05 1.14e+02 1.85e+04  -1.0 1.32e+04  -5.1 8.43e-02 8.15e-03f  1
 733  3.4498374e+05 1.11e+02 1.86e+04  -1.0 1.32e+04  -5.6 5.71e-03 2.94e-02f  1
 734  3.4495944e+05 1.11e+02 1.59e+04  -1.0 1.30e+04  -5.2 1.13e-01 2.64e-03f  1
 735  3.4401145e+05 1.04e+02 1.61e+04  -1.0 1.32e+04  -5.7 1.40e-02 7.40e-02f  1
 736  3.4395481e+05 1.03e+02 1.59e+04  -1.0 1.25e+04  -5.2 1.21e-02 5.95e-03f  1
 737  3.4383548e+05 1.02e+02 1.59e+04  -1.0 1.25e+04  -5.7 2.17e-03 1.01e-02f  1
 738  3.4087969e+05 9.31e+01 1.48e+04  -1.0 1.24e+04  -5.3 1.07e-01 2.45e-01f  1
 739  3.4082828e+05 9.27e+01 1.41e+04  -1.0 2.15e+04  -5.8 3.43e-02 3.94e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 740  3.4050990e+05 1.74e+02 1.41e+04  -1.0 1.88e+05  -6.2 1.21e-03 6.07e-03f  1
 741  3.4048820e+05 1.73e+02 8.80e+03  -1.0 2.79e+04  -5.8 2.85e-01 1.38e-03f  1
 742  3.4015501e+05 1.73e+02 8.83e+03  -1.0 5.81e+04  -6.3 2.05e-03 1.04e-02f  1
 743  3.3988724e+05 1.71e+02

...

1853  2.4572320e+04 7.59e-01 2.12e+03  -3.8 9.06e+00  -0.5 8.92e-02 1.29e-02h  1
1854  2.4569784e+04 6.86e-01 4.67e+03  -3.8 1.12e+01  -1.0 5.94e-01 9.76e-02f  1
1855  2.4567837e+04 6.74e-01 4.81e+03  -3.8 3.67e+01  -1.4 8.15e-02 1.67e-02f  1
1856  2.4547740e+04 6.38e-01 4.51e+03  -3.8 1.12e+02  -1.9 4.34e-02 5.44e-02f  1
1857  2.4547063e+04 6.37e-01 4.51e+03  -3.8 6.61e+02  -2.4 6.47e-03 5.69e-04f  1
1858  2.4525194e+04 6.33e-01 4.53e+03  -3.8 9.93e+02  -2.9 1.25e-02 7.70e-03f  1
1859  2.4524700e+04 6.32e-01 5.37e+03  -3.8 3.77e+02  -2.5 1.01e-01 5.06e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1860  2.4486518e+04 5.68e-01 4.06e+03  -3.8 1.42e+02  -2.0 1.68e-03 1.04e-01f  1
1861  2.4486056e+04 5.68e-01 4.36e+03  -3.8 4.18e+02  -2.5 3.33e-02 4.92e-04f  1
1862  2.4471628e+04 5.65e-01 4.33e+03  -3.8 1.26e+03  -3.0 2.87e-03 5.01e-03f  1
1863  2.4438626e+04 5.48e-01 4.70e+03  -3.8 4.70e+02  -2.6 8.80e-02 3.14e-02f  1
1864  2.4437474e+04 5.46e-01

1964  8.2799232e+03 3.60e+01 3.67e+03  -3.8 2.23e+03  -3.8 1.45e-05 8.31e-02f  1
1965  8.1723397e+03 3.52e+01 2.62e+03  -3.8 1.01e+03  -3.4 1.76e-01 2.60e-02f  1
1966  7.8677719e+03 2.86e+01 2.60e+03  -3.8 4.01e+02  -2.9 1.43e-03 2.07e-01f  1
1967  7.8677172e+03 2.86e+01 1.91e+03  -3.8 1.13e+03  -3.4 3.75e-02 1.13e-05f  1
1968  7.8080457e+03 2.75e+01 2.01e+03  -3.8 3.98e+02  -3.0 6.72e-04 3.90e-02f  1
1969  7.7911306e+03 2.74e+01 1.80e+03  -3.8 1.13e+03  -3.5 2.39e-02 3.47e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1970  7.7901207e+03 2.74e+01 1.70e+03  -3.8 3.52e+03  -3.9 5.41e-03 8.94e-05f  1
1971  7.7487669e+03 2.74e+01 1.52e+03  -3.8 8.32e+03  -4.4 6.14e-03 1.48e-03f  1
1972  7.7477532e+03 2.74e+01 1.39e+03  -3.8 1.57e+04  -4.9 1.66e-03 2.04e-05f  1
1973  7.5772972e+03 2.74e+01 1.47e+03  -3.8 8.03e+04  -5.4 1.40e-07 7.45e-04f  1
1974  7.5771272e+03 2.74e+01 1.41e+03  -3.8 6.94e+04  -4.9 7.66e-03 8.53e-07f  1
1975  7.1146316e+03 2.74e+01

2065  4.1375242e+03 8.92e-04 4.64e+03  -3.8 1.95e+01  -1.4 1.99e-01 2.09e-02f  1
2066  4.1375167e+03 8.90e-04 6.58e+03  -3.8 8.30e-01  -0.1 1.00e+00 2.31e-03f  1
2067  4.1349809e+03 6.04e-04 8.67e+03  -3.8 2.79e+00  -0.5 1.00e+00 3.20e-01f  1
2068  4.1296448e+03 4.54e-04 8.71e+03  -3.8 8.85e+00  -1.0 1.00e+00 2.49e-01f  1
2069  4.1282838e+03 4.44e-04 8.54e+03  -3.8 2.63e+01  -1.5 1.12e-01 2.14e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2070  4.1282725e+03 4.43e-04 1.19e+04  -3.8 9.21e-01  -0.2 1.00e+00 4.07e-03f  1
2071  4.1263467e+03 3.35e-04 9.91e+03  -3.8 2.80e+00  -0.7 1.00e+00 2.44e-01f  1
2072  4.1219283e+03 6.99e-04 8.01e+03  -3.8 8.38e+00  -1.1 1.73e-01 1.90e-01f  1
2073  4.1219173e+03 6.99e-04 8.01e+03  -3.8 2.54e+01  -1.6 5.89e-05 1.67e-04f  1
2074  4.1209026e+03 6.69e-04 8.17e+03  -3.8 9.57e+00  -1.2 2.72e-01 4.16e-02f  1
2075  4.1073288e+03 5.51e-04 6.67e+03  -3.8 2.86e+01  -1.7 2.06e-01 1.86e-01f  1
2076  4.1070054e+03 5.50e-04

2183  5.1359774e+02 6.11e+00 6.93e+05  -3.8 6.65e+02    -  5.87e-02 2.01e-02f  1
2184  4.8146439e+02 6.01e+00 6.55e+05  -3.8 6.72e+02    -  5.66e-02 6.51e-02f  1
2185  4.5723591e+02 5.89e+00 6.38e+05  -3.8 6.70e+02    -  2.91e-02 5.20e-02f  1
2186  4.3346715e+02 5.78e+00 5.67e+05  -3.8 6.74e+02    -  1.04e-01 5.32e-02f  1
2187  4.0004424e+02 5.79e+00 4.13e+05  -3.8 6.80e+02    -  2.44e-01 7.72e-02f  1
2188  3.6352647e+02 5.90e+00 4.13e+05  -3.8 6.93e+02    -  1.67e-02 8.85e-02f  1
2189  3.0639036e+02 6.86e+00 3.12e+05  -3.8 7.25e+02    -  2.26e-01 1.46e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2190  3.0635951e+02 6.86e+00 2.88e+05  -3.8 7.69e+02    -  8.26e-02 7.44e-04h  1
2191  3.0557356e+02 6.77e+00 2.87e+05  -3.8 7.93e+02    -  3.16e-03 1.18e-02f  1
2192  3.0395819e+02 6.60e+00 2.81e+05  -3.8 7.95e+02    -  2.00e-02 2.55e-02f  1
2193  3.0225974e+02 6.26e+00 2.68e+05  -3.8 8.53e+02    -  4.70e-02 5.16e-02f  1
2194  3.0193697e+02 6.21e+00

2318  6.3552649e+01 4.50e+00 7.36e+01  -3.8 7.14e+01    -  1.03e-01 3.41e-01h  1
2319  6.3551927e+01 4.29e+00 4.44e+02  -3.8 5.06e-01  -0.7 8.42e-01 4.64e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2320  6.3551672e+01 4.29e+00 5.82e+02  -3.8 3.07e+02    -  4.59e-01 3.60e-04h  1
2321  6.3549678e+01 2.86e+00 9.79e+01  -3.8 7.32e-02  -1.2 8.09e-01 3.32e-01h  1
2322  6.3549523e+01 2.85e+00 5.06e+02  -3.8 1.08e+00  -1.7 4.52e-01 5.42e-03h  1
2323  6.3556600e+01 2.63e+00 4.66e+02  -3.8 1.95e+01    -  8.01e-02 7.92e-02h  1
2324  6.3554092e+01 2.40e+00 8.02e+01  -3.8 1.50e-01  -2.1 8.15e-01 8.90e-02h  1
2325  6.3553960e+01 2.39e+00 5.68e+02  -3.8 3.11e+00  -2.6 1.82e-01 2.51e-03h  1
2326  6.3544894e+01 2.39e+00 4.76e+02  -3.8 1.92e+03    -  1.09e-02 1.87e-03h  1
2327  6.3545784e+01 2.39e+00 5.28e+02  -3.8 3.21e+02    -  5.65e-02 1.39e-03f  3
2328  6.3544842e+01 1.77e+00 1.49e+01  -3.8 2.44e-01  -3.1 1.00e+00 2.55e-01h  1
2329  6.3280170e+01 3.37e-01

In [6]:
solve_pyomo(para_truth[2])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 8.6669884e+03 9.78e+04 2.08e+03  -3.0 7.11e+01  -1.0 1.04e-01 7.03e-03f  1
 188r 8.7368752e+03 9.78e+04 2.62e+03  -3.0 6.85e+02    -  6.25e-02 2.12e-02f  1
 189r 8.9125576e+03 9.78e+04 2.77e+03  -3.0 6.71e+02    -  7.64e-02 5.45e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 8.9135836e+03 9.78e+04 2.78e+03  -3.0 1.58e+02  -1.4 1.35e-02 1.56e-03f  1
 191r 8.9144538e+03 9.78e+04 2.77e+03  -3.0 7.28e+01  -1.0 2.49e-04 3.00e-03f  1
 192r 8.9147563e+03 9.78e+04 2.77e+03  -3.0 3.00e+01  -0.6 1.05e-03 2.57e-03f  1
 193r 8.9147872e+03 9.78e+04 2.77e+03  -3.0 8.04e+01  -1.1 1.51e-05 9.62e-05f  1
 194r 8.9147874e+03 9.78e+04 2.77e+03  -3.0 1.82e+02  -0.6 2.95e-08 1.23e-06f  1
 195r 8.9147874e+03 9.78e+04 2.77e+03  -3.0 3.72e+02   0.7 7.63e-04 1.71e-07f  2
 196r 8.9179715e+03 9.78e+04 2.79e+03  -3.0 6.34e+02    -  2.75e-03 1.04e-03f  1
 197r 8.9194285e+03 9.78e+04 1.81e+03  -3.0 5.01e+00   0.2 3.60e-01 7.16e-02f  1
 198r 8.9206438e+03 9.78e+04

 498  3.9542269e+05 3.97e+03 9.79e+03  -1.0 3.98e+03  -0.4 1.15e-03 3.91e-04h  1
 499  3.9542628e+05 3.97e+03 9.79e+03  -1.0 3.97e+03  -0.9 1.38e-03 8.46e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500  3.9544081e+05 3.97e+03 9.78e+03  -1.0 3.97e+03  -1.4 2.91e-04 3.73e-04h  1
 501  3.9544169e+05 3.97e+03 9.78e+03  -1.0 3.97e+03  -0.9 2.04e-03 2.18e-05h  1
 502  3.9546605e+05 3.97e+03 9.78e+03  -1.0 3.97e+03  -1.4 3.04e-04 6.84e-04h  1
 503  3.9547945e+05 3.97e+03 1.15e+04  -1.0 3.97e+03  -1.0 7.09e-03 3.59e-04h  1
 504  3.9548736e+05 3.97e+03 1.18e+04  -1.0 3.97e+03  -1.5 1.03e-03 2.45e-04h  1
 505  3.9549020e+05 3.97e+03 1.34e+04  -1.0 3.97e+03  -1.0 1.79e-03 8.61e-05h  1
 506  3.9550126e+05 3.96e+03 1.51e+04  -1.0 3.97e+03  -1.5 4.81e-03 3.84e-04h  1
 507  3.9550559e+05 3.96e+03 1.66e+04  -1.0 3.96e+03  -1.1 1.25e-03 1.47e-04h  1
 508  3.9551709e+05 3.96e+03 1.93e+04  -1.0 4.03e+03  -1.6 6.40e-03 4.32e-04h  1
 509  3.9552224e+05 3.96e+03

 615  3.9254946e+05 1.57e+03 6.32e+04  -1.0 2.39e+04  -4.2 1.43e-02 2.17e-03f  1
 616  3.9254990e+05 1.56e+03 6.19e+04  -1.0 2.32e+04  -3.8 9.92e-04 6.15e-03h  1
 617  3.9252786e+05 1.55e+03 6.16e+04  -1.0 2.41e+04  -4.2 7.25e-03 4.06e-03f  1
 618  3.9253388e+05 1.55e+03 6.09e+04  -1.0 2.37e+04  -3.8 6.51e-04 3.82e-03h  1
 619  3.9245894e+05 1.52e+03 6.04e+04  -1.0 2.46e+04  -4.3 4.17e-03 1.72e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  3.9246726e+05 1.50e+03 6.12e+04  -1.0 2.39e+04  -3.9 2.52e-02 1.06e-02h  1
 621  3.9238105e+05 1.49e+03 6.10e+04  -1.0 2.45e+04  -4.4 2.79e-03 1.21e-02f  1
 622  3.9238135e+05 1.48e+03 6.30e+04  -1.0 2.39e+04  -3.9 3.45e-02 2.60e-03h  1
 623  3.9203525e+05 1.40e+03 6.17e+04  -1.0 2.47e+04  -4.4 2.54e-02 5.50e-02f  1
 624  3.9201199e+05 1.39e+03 6.10e+04  -1.0 2.35e+04  -4.0 3.29e-03 8.19e-03f  1
 625  3.9194272e+05 1.37e+03 5.47e+04  -1.0 2.47e+04  -4.5 8.46e-02 1.04e-02f  1
 626  3.9190855e+05 1.36e+03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 720  3.4660647e+05 6.79e+00 1.24e+02  -1.0 3.12e+03  -4.6 1.28e-02 2.19e-02f  1
 721  3.4655183e+05 6.72e+00 3.89e+02  -1.0 6.77e+03  -5.1 5.52e-02 1.12e-02f  1
 722  3.4618425e+05 5.85e+00 1.72e+02  -1.0 2.64e+03  -4.7 1.96e-03 1.49e-01f  1
 723  3.4617345e+05 5.84e+00 1.08e+02  -1.0 7.58e+03  -5.2 1.86e-02 1.94e-03f  1
 724  3.4588486e+05 6.40e+00 1.48e+02  -1.0 3.36e+03  -4.7 1.63e-02 9.79e-02f  1
 725  3.4586840e+05 6.38e+00 1.83e+02  -1.0 8.60e+03  -5.2 2.30e-02 2.46e-03f  1
 726  3.4573864e+05 6.14e+00 1.65e+02  -1.0 3.66e+03  -4.8 3.68e-02 4.09e-02f  1
 727  3.4548734e+05 6.03e+00 2.21e+02  -1.0 1.04e+04  -5.3 3.82e-02 2.75e-02f  1
 728  3.4546167e+05 5.98e+00 2.65e+02  -1.0 3.80e+03  -4.8 1.89e-02 8.13e-03f  1
 729  3.4516427e+05 6.25e+00 1.94e+02  -1.0 1.19e+04  -5.3 2.08e-03 2.74e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  3.4514326e+05 6.22e+00

...

2509  8.7777374e+02 1.16e-01 6.49e+03  -3.8 7.32e-02   1.5 2.02e-01 7.60e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2510  8.7776159e+02 1.04e-01 5.95e+03  -3.8 5.68e-02   2.0 5.76e-02 1.08e-01h  1
2511  8.7776085e+02 9.88e-02 5.54e+03  -3.8 5.24e-02   2.4 4.27e-02 4.86e-02h  1
2512  8.7775430e+02 9.15e-02 4.23e+03  -3.8 4.94e-02   1.9 1.76e-01 7.33e-02h  1
2513  8.7775404e+02 8.86e-02 5.70e+03  -3.8 4.79e-02   2.3 1.34e-01 3.15e-02h  1
2514  8.7772622e+02 6.30e-02 2.72e+03  -3.8 4.75e-02   1.9 4.00e-01 2.89e-01h  1
2515  8.7772295e+02 4.95e-02 2.77e+03  -3.8 3.60e-02   2.3 3.57e-01 2.15e-01h  1
2516  8.7769903e+02 4.05e-02 4.01e+03  -3.8 2.79e-02   1.8 7.85e-01 1.81e-01h  1
2517  8.7768929e+02 2.82e-02 5.71e+02  -3.8 2.28e-02   2.2 6.99e-01 3.05e-01h  1
2518  8.7764395e+02 1.93e-02 3.55e+02  -3.8 3.67e-02   1.8 2.72e-01 3.15e-01h  1
2519  8.7763416e+02 1.46e-02 1.54e+02  -3.8 2.46e-02   2.2 1.48e-01 2.45e-01f  1
iter    objective    inf_pr 

2618  4.5989209e+02 1.25e-02 5.00e+04  -3.8 1.55e-01  -0.4 1.00e+00 7.03e-01f  1
2619  4.5983242e+02 1.06e-02 4.55e+04  -3.8 5.34e-01  -0.9 9.90e-01 1.55e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2620  4.5861756e+02 7.27e-04 1.85e+01  -3.8 1.61e+00  -1.4 1.00e+00 1.00e+00f  1
2621  4.5643372e+02 1.56e-03 2.37e+02  -3.8 4.87e+00  -1.8 7.11e-01 5.96e-01f  1
2622  4.5630357e+02 1.53e-03 2.21e+02  -3.8 1.17e+01  -2.3 9.25e-02 1.52e-02f  1
2623  4.5603784e+02 1.49e-03 2.53e+02  -3.8 3.43e+01  -2.8 1.92e-01 1.51e-02f  1
2624  4.5550173e+02 1.55e-03 3.92e+02  -3.8 5.02e+01  -3.3 4.47e-05 1.99e-02f  1
2625  4.5549878e+02 1.54e-03 2.20e+02  -3.8 3.11e+01  -3.7 5.14e-02 1.43e-03f  1
2626  4.5531108e+02 1.59e-03 5.48e+02  -3.8 1.01e+02  -4.2 1.34e-02 8.16e-02f  1
2627  4.5471981e+02 1.56e-03 5.94e+02  -3.8 1.14e+03  -4.7 1.41e-05 9.15e-04f  1
2628  4.5458685e+02 1.66e-03 5.72e+02  -3.8 7.88e+02  -5.2 1.49e-02 8.93e-03f  1
2629  4.5408302e+02 2.31e-03

2721  2.0530768e+02 5.18e-04 5.26e+03  -3.8 3.28e-03   3.0 1.00e+00 5.09e-01h  1
2722  2.0530772e+02 4.47e-09 1.65e-01  -3.8 4.61e-04   2.6 1.00e+00 1.00e+00f  1
2723  2.0530727e+02 2.04e-09 4.28e-02  -3.8 3.59e-04   2.1 1.00e+00 1.00e+00h  1
2724  2.0530703e+02 3.01e-09 2.96e-02  -3.8 7.46e-04   1.6 1.00e+00 1.00e+00h  1
2725  2.0530668e+02 1.18e-08 2.18e-02  -3.8 1.65e-03   1.1 1.00e+00 1.00e+00f  1
2726  2.0530552e+02 1.26e-08 1.82e-02  -3.8 4.13e-03   0.6 1.00e+00 1.00e+00f  1
2727  2.0527746e+02 3.72e-05 5.10e+02  -5.7 2.23e-02   0.2 8.24e-01 8.58e-01f  1
2728  2.0526721e+02 1.33e-05 8.10e+02  -5.7 3.88e-02  -0.3 1.00e+00 6.15e-01f  1
2729  2.0523397e+02 3.31e-07 1.08e+01  -5.7 1.03e-01  -0.8 1.00e+00 9.89e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2730  2.0518319e+02 3.31e-06 1.66e+01  -5.7 3.07e-01  -1.3 1.00e+00 5.35e-01f  1
2731  2.0509416e+02 1.66e-05 1.71e+01  -5.7 9.21e-01  -1.7 1.00e+00 3.13e-01f  1
2732  2.0499129e+02 3.45e-05

2841  7.7031162e+01 5.77e-06 9.68e-01  -5.7 8.43e-04   2.2 1.00e+00 1.00e+00h  1
2842  7.7031127e+01 1.22e-09 7.24e+02  -5.7 6.74e-04   1.7 1.00e+00 7.44e-01h  1
2843  7.7031101e+01 3.07e-10 1.87e+02  -5.7 4.09e-04   1.3 1.00e+00 7.44e-01f  1
2844  7.7031114e+01 1.46e-11 7.57e-04  -5.7 1.22e-04   0.8 1.00e+00 1.00e+00f  1
2845  7.7031114e+01 1.46e-11 6.30e-04  -5.7 3.05e-04   0.3 1.00e+00 1.00e+00h  1
2846  7.7031113e+01 1.46e-11 5.00e-04  -5.7 7.26e-04  -0.2 1.00e+00 1.00e+00h  1
2847  7.7031110e+01 1.46e-11 3.78e-04  -5.7 1.65e-03  -0.6 1.00e+00 1.00e+00h  1
2848  7.7031096e+01 4.37e-11 2.79e-04  -5.7 3.65e-03  -1.1 1.00e+00 1.00e+00h  1
2849  7.7031052e+01 1.60e-10 2.14e-04  -5.7 8.40e-03  -1.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2850  7.7030967e+01 4.54e-10 1.65e-01  -5.7 2.23e-02  -2.1 1.00e+00 7.09e-01h  1
2851  7.7030658e+01 2.40e-09 1.64e-04  -5.7 5.79e-02  -2.5 1.00e+00 1.00e+00f  1
2852  7.7029762e+01 1.35e-08

2946  7.5112835e+01 1.85e+01 8.75e+02  -5.7 2.70e-01  -1.2 4.61e-02 1.82e-03h  1
2947  7.5112930e+01 1.84e+01 2.18e+03  -5.7 1.19e-01  -0.8 1.00e+00 4.82e-03h  1
2948  7.5106756e+01 1.40e+01 2.36e+03  -5.7 1.00e+02    -  2.11e-02 2.37e-01h  1
2949  7.5106778e+01 1.40e+01 1.53e+03  -5.7 4.50e-01  -1.3 1.00e+00 1.49e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2950  7.5083058e+01 9.53e+00 2.63e+03  -5.7 1.99e+02    -  5.62e-02 3.19e-01h  1
2951  7.5085932e+01 6.74e+00 7.26e+02  -5.7 5.39e-02  -0.8 1.00e+00 2.92e-01h  1
2952  7.5092992e+01 3.68e-08 7.58e-01  -5.7 4.88e-03  -0.4 1.00e+00 1.00e+00h  1
2953  7.5093018e+01 2.16e-09 2.87e+01  -5.7 1.98e-02  -0.9 6.74e-01 9.42e-01H  1
2954  7.5093008e+01 4.19e-10 5.97e+00  -5.7 6.75e-03  -0.5 1.00e+00 8.05e-01f  1
2955  7.5093001e+01 4.12e-10 1.65e+02  -5.7 1.25e-01  -0.9 2.85e-01 1.77e-02f  4
2956  7.5092960e+01 6.87e-10 5.71e-01  -5.7 1.01e-02  -0.5 1.00e+00 1.00e+00h  1
2957  7.5092934e+01 7.48e-11

In [7]:
solve_pyomo(para_truth[3])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 9.2410761e+03 9.74e+04 1.71e+03  -4.5 1.67e+02  -0.6 9.26e-01 2.92e-02f  1
 188r 1.0285165e+04 9.70e+04 1.24e+03  -4.5 4.89e+02  -1.1 1.53e-01 6.70e-01f  1
 189r 1.0868879e+04 9.69e+04 4.71e+02  -4.5 1.83e+02  -0.7 8.53e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.0869065e+04 9.69e+04 8.73e+02  -4.5 5.24e+02  -1.2 2.05e-01 1.08e-04f  1
 191r 1.1293346e+04 9.67e+04 7.80e+02  -4.5 2.02e+02  -0.7 1.00e+00 6.51e-01f  1
 192r 1.1293433e+04 9.67e+04 2.94e+03  -4.5 5.79e+02  -1.2 3.60e-01 4.59e-05f  1
 193r 1.1621476e+04 9.66e+04 1.39e+03  -4.5 2.24e+02  -0.8 1.00e+00 4.51e-01f  1
 194r 1.2354117e+04 9.64e+04 1.96e+03  -4.5 6.41e+02  -1.3 5.13e-01 3.45e-01f  1
 195r 1.3861054e+04 9.60e+04 6.49e+03  -4.5 1.71e+03  -1.8 5.09e-01 2.55e-01f  1
 196r 1.3981808e+04 9.59e+04 6.04e+03  -4.5 6.77e+02  -1.3 3.00e-04 5.26e-02f  1
 197r 1.4299699e+04 9.58e+04 2.03e+03  -4.5 2.63e+02  -0.9 1.00e+00 3.61e-01f  1
 198r 1.6375086e+04 9.52e+04

 482  2.8563538e+05 1.78e+02 1.24e+06  -1.0 6.97e+02  -0.9 2.33e-02 8.23e-03h  1
 483  2.8563334e+05 1.77e+02 1.21e+06  -1.0 6.83e+02  -1.4 3.38e-02 6.40e-03h  1
 484  2.8563207e+05 1.76e+02 1.23e+06  -1.0 6.85e+02  -1.0 5.00e-02 6.17e-03h  1
 485  2.8563123e+05 1.75e+02 1.22e+06  -1.0 6.91e+02  -1.5 1.58e-02 5.29e-03h  1
 486  2.8563143e+05 1.74e+02 1.24e+06  -1.0 6.99e+02  -1.0 3.89e-02 6.05e-03h  1
 487  2.8563158e+05 1.73e+02 1.19e+06  -1.0 6.97e+02  -1.5 6.49e-02 6.59e-03h  1
 488  2.8563250e+05 1.72e+02 1.19e+06  -1.0 7.15e+02  -1.1 3.64e-03 3.43e-03h  1
 489  2.8563600e+05 1.70e+02 1.17e+06  -1.0 7.19e+02  -1.6 1.43e-02 1.33e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  2.8563733e+05 1.69e+02 1.22e+06  -1.0 7.05e+02  -1.1 5.47e-02 4.27e-03h  1
 491  2.8563881e+05 1.68e+02 1.20e+06  -1.0 7.04e+02  -1.6 4.16e-02 4.09e-03h  1
 492  2.8564196e+05 1.68e+02 1.19e+06  -1.0 7.27e+02  -1.2 3.20e-03 4.06e-03h  1
 493  2.8565416e+05 1.66e+02

 588  2.8543260e+05 3.14e+01 3.16e+05  -1.0 5.71e+02  -1.8 3.00e-03 7.09e-03h  1
 589  2.8543265e+05 3.14e+01 3.38e+05  -1.0 5.47e+02  -1.4 5.19e-03 1.38e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 590  2.8543199e+05 3.12e+01 3.36e+05  -1.0 5.16e+02  -1.9 3.26e-03 5.37e-03h  1
 591  2.8543080e+05 3.08e+01 3.21e+05  -1.0 5.10e+02  -1.5 1.03e-02 1.27e-02h  1
 592  2.8543065e+05 3.08e+01 2.71e+05  -1.0 5.35e+02  -1.9 1.47e-01 1.32e-03h  1
 593  2.8542000e+05 2.73e+01 1.43e+05  -1.0 5.05e+02  -1.5 1.05e-02 1.15e-01f  1
 594  2.8541978e+05 2.73e+01 1.42e+05  -1.0 4.37e+02  -2.0 1.65e-03 2.15e-03h  1
 595  2.8541124e+05 2.59e+01 8.11e+05  -1.0 4.45e+02  -2.5 3.09e-01 5.16e-02f  1
 596  2.8536760e+05 1.61e+01 1.79e+05  -1.0 4.13e+02  -2.1 3.59e-02 3.99e-01f  1
 597  2.8534808e+05 1.41e+01 8.27e+05  -1.0 2.67e+02  -2.5 4.55e-01 1.26e-01f  1
 598  2.8534797e+05 1.41e+01 8.35e+05  -1.0 2.27e+02  -2.1 1.57e-02 1.43e-03h  1
 599  2.8534814e+05 1.41e+01

 693  2.5208613e+05 1.05e+01 1.46e+05  -1.0 1.13e+03  -4.1 5.56e-02 1.78e-02f  1
 694  2.5194648e+05 9.84e+00 1.55e+04  -1.0 5.86e+02  -3.7 1.50e-02 7.79e-02f  1
 695  2.5162130e+05 1.02e+01 3.07e+04  -1.0 4.89e+03  -4.1 7.68e-04 1.94e-02f  1
 696  2.5161689e+05 1.02e+01 9.63e+04  -1.0 8.86e+02  -3.7 5.94e-02 1.20e-03f  1
 697  2.5125442e+05 1.02e+01 3.08e+04  -1.0 3.21e+03  -4.2 1.98e-04 3.14e-02f  1
 698  2.5121939e+05 1.01e+01 1.50e+05  -1.0 1.73e+03  -3.8 6.47e-02 6.46e-03f  1
 699  2.5121898e+05 9.84e+00 6.48e+05  -1.0 2.53e+01  -1.5 2.77e-01 2.94e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 700  2.5122179e+05 9.05e+00 7.98e+05  -1.0 1.01e+01  -1.1 1.86e-01 8.03e-02f  1
 701  2.5122424e+05 5.82e+00 8.04e+05  -1.0 1.80e+01  -1.6 5.05e-01 3.55e-01f  1
 702  2.5122236e+05 5.69e+00 2.36e+06  -1.0 4.09e+01  -2.1 7.44e-01 2.13e-02f  1
 703  2.5117761e+05 4.62e+00 3.17e+06  -1.0 6.62e+01  -2.5 1.00e+00 1.90e-01f  1
 704  2.5078420e+05 5.85e+00

...

1264  7.0233728e+04 7.29e+00 4.10e+04  -5.7 2.31e+02  -2.6 8.27e-02 1.85e-01f  1
1265  6.9895546e+04 6.53e+00 3.78e+04  -5.7 7.10e+02  -3.1 1.23e-01 1.81e-01f  1
1266  6.9895180e+04 6.53e+00 1.73e+04  -5.7 2.65e+02  -2.7 3.31e-01 5.48e-04f  1
1267  6.9171835e+04 2.22e+01 2.72e+04  -5.7 8.50e+02  -3.2 1.45e-06 3.23e-01f  1
1268  6.9159189e+04 2.19e+01 4.30e+03  -5.7 3.47e+02  -2.7 3.99e-01 1.40e-02f  1
1269  6.8935825e+04 7.66e+00 1.32e+04  -5.7 1.32e+02  -2.3 3.68e-01 6.95e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1270  6.8601081e+04 9.08e+00 1.65e+04  -5.7 3.96e+02  -2.8 3.57e-04 3.14e-01f  1
1271  6.8566481e+04 8.35e+00 1.01e+04  -5.7 1.55e+02  -2.4 2.54e-01 8.38e-02f  1
1272  6.8566409e+04 8.35e+00 7.86e+03  -5.7 4.21e+02  -2.8 3.51e-01 6.30e-05f  1
1273  6.8306956e+04 7.75e+00 7.02e+03  -5.7 1.20e+03  -3.3 3.21e-01 7.93e-02f  1
1274  6.8277802e+04 7.73e+00 7.29e+03  -5.7 3.41e+03  -3.8 4.86e-02 3.12e-03f  1
1275  6.7854012e+04 7.56e+00

1369  2.6318232e+03 3.10e+01 3.89e+04  -5.7 1.99e+03  -3.4 9.70e-05 8.07e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  2.6318082e+03 3.10e+01 3.97e+04  -5.7 8.10e+02  -3.9 1.59e-02 1.12e-05h  1
1371  2.6316220e+03 3.10e+01 3.97e+04  -5.7 1.21e+03  -4.4 2.13e-08 9.39e-05f  1
1372  2.6153719e+03 3.08e+01 3.94e+04  -5.7 1.43e+03  -4.9 8.30e-03 7.52e-03f  1
1373  2.6105035e+03 3.07e+01 3.94e+04  -5.7 2.01e+03  -5.4 1.29e-08 1.41e-03f  1
1374  2.6035867e+03 3.06e+01 3.94e+04  -5.7 4.68e+02  -3.1 1.02e-03 3.98e-03f  1
1375  2.5886401e+03 3.06e+01 3.95e+04  -5.7 2.29e+03  -3.6 1.16e-03 1.80e-03f  1
1376  2.5526688e+03 3.01e+01 3.94e+04  -5.7 5.34e+02  -3.2 8.22e-03 1.83e-02f  1
1377  2.5524908e+03 3.01e+01 3.66e+04  -5.7 1.57e+03  -3.6 2.82e-02 3.41e-05f  1
1378  2.4644931e+03 3.02e+01 3.77e+04  -5.7 3.86e+03  -4.1 2.56e-04 8.69e-03f  1
1379  2.4227645e+03 3.00e+01 3.72e+04  -5.7 3.28e+03  -4.6 8.89e-03 6.69e-03f  1
iter    objective    inf_pr 

1468  4.4892790e+02 1.92e+01 1.96e+04  -5.7 8.28e-01   2.6 9.34e-05 7.34e-04h  1
1469  4.4892060e+02 1.90e+01 1.98e+04  -5.7 1.14e+00   2.1 4.12e-03 7.22e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  4.4891915e+02 1.90e+01 1.96e+04  -5.7 8.90e-01   2.6 1.75e-03 1.11e-03h  1
1471  4.4891459e+02 1.89e+01 1.99e+04  -5.7 1.07e+00   2.1 1.01e-03 4.23e-03h  1
1472  4.4891349e+02 1.89e+01 1.98e+04  -5.7 8.37e-01   2.5 1.17e-03 8.22e-04h  1
1473  4.4890795e+02 1.88e+01 2.00e+04  -5.7 1.09e+00   2.0 1.61e-03 5.16e-03h  1
1474  4.4890695e+02 1.88e+01 1.98e+04  -5.7 8.41e-01   2.5 1.54e-03 7.48e-04h  1
1475  4.4890019e+02 1.87e+01 2.01e+04  -5.7 1.09e+00   2.0 6.68e-04 6.32e-03h  1
1476  4.4889831e+02 1.86e+01 1.74e+04  -5.7 8.28e-01   2.4 1.40e-02 1.36e-03f  1
1477  4.4889753e+02 1.86e+01 1.69e+04  -5.7 1.13e+00   1.9 7.01e-03 7.09e-04h  1
1478  4.4889075e+02 1.85e+01 1.78e+04  -5.7 9.17e-01   2.3 6.57e-04 5.21e-03h  1
1479  4.4888924e+02 1.85e+01

1585  2.0697248e+02 3.75e-01 5.89e+03  -5.7 4.15e-01  -0.8 2.69e-01 1.59e-01h  1
1586  2.0694134e+02 3.70e-01 6.15e+03  -5.7 1.28e+00  -1.2 1.08e-01 1.38e-02h  1
1587  2.0674171e+02 2.86e-01 4.32e+03  -5.7 4.05e-01  -0.8 1.11e-01 2.25e-01h  1
1588  2.0669698e+02 2.80e-01 4.33e+03  -5.7 1.37e+00  -1.3 4.74e-02 1.96e-02h  1
1589  2.0644309e+02 2.73e-01 4.37e+03  -5.7 1.47e+02  -1.8 1.04e-03 1.74e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1590  2.0634657e+02 2.63e-01 4.19e+03  -5.7 1.83e+00  -1.3 2.73e-02 3.57e-02f  1
1591  2.0630295e+02 2.51e-01 4.41e+03  -5.7 4.77e-01  -0.9 1.70e-01 4.67e-02h  1
1592  2.0612921e+02 2.35e-01 4.26e+03  -5.7 2.19e+00  -1.4 1.05e-01 5.54e-02f  1
1593  2.0566090e+02 1.17e-01 3.37e+03  -5.7 5.45e-01  -1.0 8.70e-01 4.59e-01f  1
1594  2.0495639e+02 5.47e-02 2.58e+03  -5.7 2.00e+00  -1.4 2.42e-01 2.33e-01f  1
1595  2.0464831e+02 3.07e-02 2.03e+03  -5.7 5.50e-01  -1.0 7.08e-01 3.04e-01f  1
1596  2.0419401e+02 7.66e-03

1699  1.4863821e+02 1.26e-08 1.76e-03  -5.7 9.62e-02  -1.7 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1700  1.4860599e+02 1.74e-07 2.46e+00  -5.7 2.84e-01  -2.2 4.67e-01 1.00e+00f  1
1701  1.4850887e+02 1.71e-06 1.74e-03  -5.7 8.59e-01  -2.7 1.00e+00 1.00e+00f  1
1702  1.4821722e+02 3.91e-05 3.12e+01  -5.7 2.58e+00  -3.2 3.38e-01 1.00e+00f  1
1703  1.4735786e+02 9.54e-04 6.35e+01  -5.7 7.76e+00  -3.6 6.16e-01 9.79e-01f  1
1704  1.4471451e+02 2.55e-02 3.02e+02  -5.7 2.33e+01  -4.1 3.85e-01 1.00e+00f  1
1705  1.4265673e+02 5.04e-02 1.44e+02  -5.7 6.83e+01  -4.6 5.01e-01 2.66e-01f  1
1706  1.2686727e+02 4.45e+00 1.57e+03  -5.7 1.85e+02  -5.1 2.65e-01 7.59e-01f  1
1707  1.0659548e+02 1.73e+01 4.93e+02  -5.7 2.62e+02  -5.6 8.24e-01 8.82e-01f  1
1708  1.0659544e+02 1.73e+01 7.65e+02  -5.7 4.60e+02  -6.0 2.78e-01 1.57e-06h  1
1709  1.0659543e+02 1.73e+01 2.56e+03  -5.7 4.00e+02  -6.5 2.09e-01 3.29e-07h  2
iter    objective    inf_pr 

In [8]:
solve_pyomo(para_truth[4])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 315r 4.1316656e+04 9.00e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 4.25e-09R  2
 316r 4.1316451e+04 9.00e+04 9.99e+02   5.0 1.30e+09    -  1.94e-09 1.02e-10f  1
 317r 4.1277425e+04 9.00e+04 9.99e+02   2.9 3.46e+06    -  4.31e-06 4.08e-06f  1
 318r 4.1517890e+04 8.99e+04 9.99e+02   2.9 1.06e+06    -  3.51e-05 2.25e-04f  1
 319r 4.1771353e+04 8.99e+04 9.99e+02   2.9 3.11e+05    -  1.34e-03 2.20e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320r 4.3368272e+04 8.94e+04 1.89e+03   2.9 3.07e+05    -  5.25e-03 1.38e-03f  1
 321r 4.6847026e+04 8.85e+04 3.53e+03   2.9 3.81e+05    -  5.43e-03 2.38e-03f  1
 322r 4.8672189e+04 8.81e+04 4.67e+03   2.9 5.13e+05    -  2.33e-03 9.17e-04f  1
 323r 5.0809370e+04 8.75e+04 5.42e+03   2.9 5.47e+05    -  1.71e-03 1.01e-03f  1
 324r 5.2110872e+04 8.72e+04 5.95e+03   2.9 5.28e+05    -  1.00e-03 6.33e-04f  1
 325r 5.3704812e+04 8.68e+04 6.81e+03   2.9 5.69e+05    -  1.26e-03 7.12e-04f  1
 326r 5.5829830e+04 8.62e+04

 559  4.1645680e+05 5.35e+02 1.71e+04  -1.0 3.65e+03  -0.8 3.38e-04 8.78e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560  4.1645659e+05 5.35e+02 1.79e+04  -1.0 5.37e+03  -1.3 2.64e-04 2.57e-05h  1
 561r 4.1645659e+05 5.35e+02 9.99e+02   2.7 0.00e+00  -0.9 0.00e+00 2.85e-07R  5
 562r 4.1646735e+05 5.35e+02 4.27e+03   2.7 3.57e+05    -  3.23e-03 1.74e-05f  1
 563r 4.1619474e+05 5.25e+02 9.18e+04   2.7 8.43e+04    -  7.72e-02 8.02e-03f  1
 564r 4.1621161e+05 4.63e+02 8.62e+04   2.0 6.35e+04    -  6.54e-02 4.38e-02f  1
 565  4.1621342e+05 4.62e+02 1.29e+04  -1.0 4.63e+02  -1.3 1.13e-02 9.85e-04h  1
 566  4.1621368e+05 4.62e+02 1.29e+04  -1.0 4.62e+02  -1.8 3.52e-03 7.93e-04h  1
 567  4.1621371e+05 4.62e+02 1.29e+04  -1.0 4.62e+02  -1.4 1.49e-02 1.08e-04h  1
 568  4.1621135e+05 4.61e+02 1.29e+04  -1.0 1.18e+03  -1.9 3.83e-03 7.48e-04f  1
 569  4.1620958e+05 4.61e+02 1.29e+04  -1.0 2.27e+03  -2.3 3.91e-03 2.44e-04f  1
iter    objective    inf_pr 

 674  3.9956250e+05 1.67e+02 4.15e+05  -1.0 2.35e+04  -2.8 3.31e-03 9.88e-04f  1
 675  3.9942866e+05 1.65e+02 4.05e+05  -1.0 2.38e+04  -3.2 1.53e-04 1.36e-02f  1
 676  3.9935557e+05 1.64e+02 4.02e+05  -1.0 2.39e+04  -3.7 8.44e-03 6.39e-03f  1
 677  3.9926103e+05 1.63e+02 3.89e+05  -1.0 2.53e+04  -4.2 2.10e-02 5.41e-03f  1
 678  3.9922042e+05 1.63e+02 3.78e+05  -1.0 2.66e+04  -4.7 1.50e-02 1.23e-03f  1
 679  3.9920679e+05 1.63e+02 3.68e+05  -1.0 2.44e+04  -4.3 1.47e-02 8.13e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 680  3.9869195e+05 1.60e+02 3.74e+05  -1.0 2.43e+04  -4.7 1.06e-04 1.60e-02f  1
 681  3.9805950e+05 1.55e+02 3.84e+05  -1.0 2.48e+04  -4.3 3.25e-04 3.19e-02f  1
 682  3.9804685e+05 1.55e+02 3.39e+05  -1.0 2.39e+04  -4.8 6.05e-02 3.36e-04f  1
 683  3.9801729e+05 1.54e+02 3.38e+05  -1.0 2.39e+04  -3.4 9.11e-04 2.78e-03f  1
 684  3.9800859e+05 1.54e+02 3.37e+05  -1.0 2.52e+04  -3.9 2.12e-03 3.67e-04f  1
 685  3.9633191e+05 1.41e+02

 785  3.3275307e+05 2.41e+01 1.40e+04  -1.0 1.68e+01  -0.6 2.79e-01 2.03e-01h  1
 786  3.3275469e+05 2.37e+01 1.38e+04  -1.0 5.00e+01  -1.1 1.43e-02 1.75e-02h  1
 787  3.3282732e+05 7.12e+00 5.45e+03  -1.0 1.29e+01  -0.7 5.43e-01 7.01e-01h  1
 788  3.3283106e+05 5.99e+00 2.18e+04  -1.0 4.36e+00  -1.2 9.11e-01 1.58e-01h  1
 789  3.3284583e+05 1.67e+00 3.85e+03  -1.0 8.36e+00  -1.6 3.59e-01 7.22e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 790  3.3284456e+05 1.46e+00 3.66e+04  -1.0 1.26e+01  -2.1 1.00e+00 1.25e-01f  1
 791  3.3283331e+05 7.80e-01 2.28e+04  -1.0 2.66e+01  -2.6 5.37e-01 4.69e-01f  1
 792  3.3278972e+05 5.84e-01 2.69e+04  -1.0 7.30e+01  -3.1 5.95e-01 4.44e-01f  1
 793  3.3267651e+05 1.78e+00 1.13e+05  -1.0 2.65e+02  -3.6 1.00e+00 4.17e-01f  1
 794  3.3232935e+05 6.59e+00 9.57e+03  -1.0 6.89e+02  -4.0 2.40e-01 4.35e-01f  1
 795  3.3215847e+05 8.05e+00 1.61e+04  -1.0 2.03e+03  -4.5 8.95e-02 7.44e-02f  1
 796  3.3194651e+05 1.06e+01

...

2368  1.3369199e+02 9.10e-01 3.15e+03  -3.8 8.17e+00  -1.0 5.54e-07 2.91e-07f  2
2369  1.3369199e+02 9.10e-01 3.15e+03  -3.8 9.30e+00   1.2 6.59e-03 2.96e-07f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2370  1.3359463e+02 6.77e-01 2.72e+03  -3.8 6.25e-02   0.8 1.00e+00 2.56e-01h  1
2371  1.3353361e+02 5.87e-01 5.12e+03  -3.8 9.66e-02   0.3 6.56e-01 1.33e-01h  1
2372  1.3312260e+02 2.88e-01 8.96e+02  -3.8 2.81e-01  -0.2 2.84e-02 5.77e-01h  1
2373  1.3304417e+02 4.81e-04 1.83e+00  -3.8 1.63e-02   1.1 1.00e+00 1.00e+00h  1
2374  1.3302834e+02 1.81e-06 5.91e-01  -3.8 5.21e-03   1.6 1.00e+00 1.00e+00h  1
2375  1.3300587e+02 2.73e-04 2.14e+01  -3.8 4.49e-02   1.1 3.68e-01 4.59e-01f  1
2376  1.3299028e+02 4.18e-04 7.14e+02  -3.8 8.02e-02   0.6 1.00e+00 8.49e-02f  1
2377  1.3252120e+02 1.17e-01 1.13e+02  -3.8 1.40e-01   0.1 5.77e-01 1.00e+00f  1
2378  1.3149089e+02 7.42e-01 8.67e+02  -3.8 5.70e-01  -0.3 9.02e-01 4.97e-01f  1
2379  1.3070102e+02 1.19e+00

2474  1.0162419e+02 1.07e-04 3.96e-03  -5.7 7.26e-01  -2.3 1.00e+00 1.00e+00f  1
2475  1.0158561e+02 1.37e-04 2.46e+01  -5.7 2.17e+00  -2.7 5.87e-01 2.34e-01f  1
2476  1.0158561e+02 1.30e-04 3.18e+02  -5.7 5.95e-03   2.2 5.84e-01 4.70e-02h  1
2477  1.0158561e+02 9.84e-05 1.96e+01  -5.7 7.31e-05   1.7 1.00e+00 2.32e-01f  2
2478  1.0158563e+02 1.46e-11 5.06e-03  -5.7 2.87e-04   1.2 1.00e+00 1.00e+00h  1
2479  1.0158558e+02 1.46e-11 3.64e-03  -5.7 6.19e-04   0.8 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2480  1.0158544e+02 1.46e-11 3.60e-03  -5.7 1.84e-03   0.3 1.00e+00 1.00e+00f  1
2481  1.0158502e+02 1.46e-11 3.59e-03  -5.7 5.50e-03  -0.2 1.00e+00 1.00e+00f  1
2482  1.0158377e+02 4.82e-11 3.59e-03  -5.7 1.65e-02  -0.7 1.00e+00 1.00e+00f  1
2483  1.0158001e+02 4.27e-10 3.59e-03  -5.7 4.95e-02  -1.1 1.00e+00 1.00e+00f  1
2484  1.0156875e+02 3.84e-09 3.59e-03  -5.7 1.49e-01  -1.6 1.00e+00 1.00e+00f  1
2485  1.0153498e+02 4.42e-05

2574  8.7598454e+01 2.65e-01 3.73e+03  -5.7 8.50e-03   1.9 1.00e+00 1.01e-04h  1
2575  8.7605069e+01 2.02e-01 4.72e+03  -5.7 6.95e-03   1.4 1.00e+00 2.35e-01h  1
2576  8.7618649e+01 7.51e-02 2.16e+02  -5.7 6.59e-03   0.9 1.00e+00 6.29e-01h  1
2577  8.7626690e+01 1.46e-06 3.30e-02  -5.7 2.46e-03   0.5 1.00e+00 1.00e+00h  1
2578  8.7626681e+01 1.46e-11 1.13e-03  -5.7 1.19e-03  -0.0 1.00e+00 1.00e+00h  1
2579  8.7626613e+01 4.37e-11 8.56e-04  -5.7 2.69e-03  -0.5 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2580  8.7626420e+01 4.37e-11 6.16e-01  -5.7 6.95e-03  -1.0 1.00e+00 8.45e-01f  1
2581  8.7626120e+01 1.96e-10 1.38e+00  -5.7 2.01e-02  -1.5 1.00e+00 4.71e-01f  1
2582  8.7624351e+01 3.97e-09 1.32e-01  -5.7 5.57e-02  -1.9 1.00e+00 9.50e-01f  1
2583  8.7623468e+01 4.44e-09 2.75e+00  -5.7 1.77e-01  -2.4 1.00e+00 1.70e-01f  1
2584  8.7608012e+01 4.16e-05 6.06e-04  -5.7 4.63e-01  -2.9 1.00e+00 1.00e+00f  1
2585  8.7561836e+01 4.98e-04

2681  7.3917788e+01 2.03e+01 1.61e+03  -5.7 5.75e+02    -  2.19e-02 3.95e-01f  1
2682  7.3939470e+01 1.81e+01 9.40e+02  -5.7 9.41e-02   0.1 5.08e-01 1.13e-01h  1
2683  7.3936761e+01 1.80e+01 9.44e+02  -5.7 1.06e+04    -  2.25e-07 2.16e-04h  1
2684  7.3938552e+01 1.77e+01 9.13e+02  -5.7 2.04e-01  -0.4 3.47e-02 1.78e-02h  1
2685  7.3938560e+01 1.77e+01 9.13e+02  -5.7 3.27e-01  -0.9 1.38e-04 7.64e-05h  1
2686  7.3938562e+01 1.77e+01 9.13e+02  -5.7 3.83e-01  -1.4 1.31e-04 2.09e-05h  1
2687  7.3938564e+01 1.77e+01 5.10e+03  -5.7 1.89e-01  -1.8 1.00e+00 2.00e-05h  1
2688  7.3912492e+01 1.70e+01 5.01e+03  -5.7 2.34e+02    -  1.34e-02 4.27e-02h  1
2689  7.3912767e+01 1.69e+01 5.00e+03  -5.7 1.74e+00  -2.3 3.56e-04 2.25e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2690  7.3913074e+01 1.69e+01 4.60e+03  -5.7 1.75e-01  -2.8 7.68e-02 1.91e-03h  1
2691  7.3913104e+01 1.69e+01 4.60e+03  -5.7 1.52e+01  -3.3 2.08e-03 1.93e-04h  1
2692  7.4029135e+01 4.97e+00

2805  7.2587124e+01 3.17e-10 3.48e-01  -5.7 1.04e-02  -1.2 1.00e+00 1.00e+00h  1
2806  7.2587113e+01 3.95e-10 2.01e+02  -5.7 4.67e-02  -1.7 1.00e+00 1.78e-01h  2
2807  7.2587097e+01 3.02e-09 3.41e+02  -5.7 1.10e+00    -  5.95e-01 8.29e-02h  2
2808  7.2586069e+01 1.03e+00 2.60e+02  -5.7 1.15e+04    -  3.52e-03 5.77e-03h  1
2809  7.2583238e+01 1.92e+01 1.02e+03  -5.7 2.31e+02    -  2.52e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2810  7.2584603e+01 5.86e-01 1.34e-01  -5.7 1.22e+01    -  1.00e+00 1.00e+00h  1
2811  7.2584683e+01 5.16e-08 4.07e-03  -5.7 4.25e-01    -  1.00e+00 1.00e+00h  1
2812  7.2584683e+01 1.46e-11 5.86e-06  -5.7 6.54e-03    -  1.00e+00 1.00e+00h  1
2813  7.2584239e+01 6.31e-03 3.20e+00  -8.6 3.23e+00    -  9.91e-01 9.74e-01h  1
2814  7.2584228e+01 3.99e-09 1.90e-04  -8.6 1.87e-01    -  1.00e+00 1.00e+00h  1
2815  7.2584228e+01 1.46e-11 5.50e-10  -8.6 9.36e-04    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 2

In [9]:
solve_pyomo(para_truth[5])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 186r 2.0163319e+04 9.44e+04 3.28e+02  -6.7 7.39e+02  -1.2 2.82e-01 4.45e-01f  1
 187r 2.0163319e+04 9.44e+04 2.11e+03  -6.7 2.77e+02  -0.8 1.00e+00 2.20e-07f  1
 188r 2.0175414e+04 9.44e+04 5.13e+03  -6.7 8.30e+02  -1.3 6.26e-01 4.92e-03f  1
 189r 2.0941835e+04 9.41e+04 1.95e+03  -6.7 3.12e+02  -0.9 4.20e-01 8.30e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 2.1861043e+04 9.38e+04 5.42e+03  -6.7 9.32e+02  -1.3 1.00e+00 3.33e-01f  1
 191r 2.2897923e+04 9.35e+04 1.83e+03  -6.7 3.50e+02  -0.9 2.41e-01 1.00e+00f  1
 192r 2.3343853e+04 9.33e+04 1.95e+03  -6.7 1.04e+03  -1.4 1.93e-01 1.44e-01f  1
 193r 2.3555550e+04 9.33e+04 2.44e+03  -6.7 3.93e+02  -1.0 7.27e-01 1.82e-01f  1
 194r 2.3555550e+04 9.33e+04 1.30e+03  -6.7 1.47e+02  -0.5 1.00e+00 1.28e-07f  1
 195r 2.4634940e+04 9.29e+04 6.96e+02  -6.7 4.41e+02  -1.0 1.00e+00 8.25e-01f  1
 196r 2.5938488e+04 9.24e+04 6.49e+03  -6.7 1.31e+03  -1.5 8.34e-01 3.34e-01f  1
 197r 2.7324101e+04 9.20e+04

 422  1.6153576e+05 8.27e+02 7.17e+03  -1.0 2.71e+03  -0.8 4.31e-04 4.57e-04h  1
 423  1.6153615e+05 8.27e+02 8.57e+03  -1.0 1.17e+03  -0.3 6.81e-03 1.83e-05h  1
 424  1.6155671e+05 8.26e+02 8.56e+03  -1.0 1.34e+03  -0.8 1.18e-03 1.19e-03h  1
 425  1.6157300e+05 8.25e+02 1.10e+04  -1.0 1.30e+03  -0.4 1.07e-02 8.48e-04h  1
 426  1.6158312e+05 8.24e+02 1.19e+04  -1.0 1.42e+03  -0.9 1.12e-02 6.55e-04h  1
 427  1.6158437e+05 8.24e+02 1.44e+04  -1.0 1.58e+03  -0.4 7.34e-03 8.82e-05h  1
 428  1.6158829e+05 8.24e+02 1.52e+04  -1.0 1.54e+03  -0.0 1.16e-03 2.54e-04h  1
 429  1.6161182e+05 8.23e+02 1.46e+04  -1.0 1.71e+03  -0.5 4.81e-04 1.96e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430  1.6161252e+05 8.22e+02 2.26e+04  -1.0 1.19e+03   0.8 2.37e-03 3.35e-05h  1
 431  1.6162109e+05 8.22e+02 7.74e+04  -1.0 1.56e+03   0.4 1.15e-02 5.74e-04h  1
 432  1.6163383e+05 8.21e+02 7.66e+04  -1.0 1.78e+03  -0.1 9.99e-04 1.22e-03h  1
 433  1.6163795e+05 8.21e+02

 535  1.6183203e+05 1.56e+02 1.61e+05  -1.0 7.40e+02  -2.3 1.44e-02 2.57e-02h  1
 536  1.6184333e+05 1.51e+02 1.36e+05  -1.0 7.15e+02  -1.9 3.62e-03 3.63e-02h  1
 537  1.6181480e+05 1.25e+02 2.01e+05  -1.0 1.09e+03  -2.3 3.13e-02 1.68e-01f  1
 538  1.6181591e+05 1.25e+02 2.01e+05  -1.0 6.70e+02  -1.9 8.48e-02 6.32e-03h  1
 539  1.6178997e+05 1.13e+02 2.14e+05  -1.0 9.74e+02  -2.4 4.83e-02 9.39e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  1.6178959e+05 1.13e+02 1.82e+05  -1.0 1.06e+04  -2.9 8.17e-03 1.13e-04h  1
 541  1.6177612e+05 1.07e+02 2.06e+05  -1.0 7.87e+02  -2.4 8.37e-04 5.38e-02f  1
 542  1.6178198e+05 9.69e+01 1.87e+05  -1.0 6.33e+02  -2.0 9.19e-02 9.25e-02h  1
 543  1.6174962e+05 9.05e+01 1.66e+05  -1.0 9.97e+02  -2.5 7.72e-02 6.58e-02f  1
 544  1.6174962e+05 9.03e+01 1.36e+05  -1.0 5.01e+02  -2.1 8.59e-02 1.97e-03h  1
 545  1.6172778e+05 8.56e+01 7.79e+04  -1.0 6.09e+02  -2.5 1.44e-01 5.20e-02f  1
 546  1.6162769e+05 8.06e+01

 649  9.4569973e+04 6.27e-04 2.02e+03  -2.5 4.61e+00  -0.7 7.03e-01 1.53e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  9.4564556e+04 9.17e-04 2.57e+03  -2.5 1.30e+01  -1.2 1.00e+00 1.36e-01f  1
 651  9.4550024e+04 7.06e-03 2.27e+03  -2.5 3.79e+01  -1.6 2.40e-01 1.23e-01f  1
 652  9.4544180e+04 5.01e-05 9.37e-01  -2.5 1.94e+00  -0.3 1.00e+00 1.00e+00f  1
 653  9.4532557e+04 3.59e-04 2.94e+02  -2.5 5.33e+00  -0.8 1.00e+00 7.02e-01f  1
 654  9.4527233e+04 4.07e-04 5.56e+02  -2.5 1.54e+01  -1.3 1.00e+00 1.08e-01f  1
 655  9.4506619e+04 3.58e-04 6.85e+02  -2.5 4.53e+01  -1.7 1.00e+00 1.42e-01f  1
 656  9.4450021e+04 1.33e-03 7.26e+02  -2.5 1.31e+02  -2.2 5.38e-01 1.31e-01f  1
 657  9.3952020e+04 8.32e-02 4.47e+02  -2.5 3.84e+02  -2.7 3.77e-01 3.83e-01f  1
 658  9.3951384e+04 7.70e-02 2.49e+02  -2.5 2.57e+00  -0.5 4.83e-01 7.39e-02f  1
 659  9.3949236e+04 7.01e-02 2.38e+02  -2.5 7.10e+00  -0.9 1.00e+00 9.02e-02f  1
iter    objective    inf_pr 

 759  5.2496449e+04 5.42e+01 5.03e+02  -2.5 5.60e+01  -0.8 4.63e-04 2.14e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  5.2495201e+04 5.34e+01 8.07e+02  -2.5 4.12e+01  -1.3 7.03e-03 1.52e-02f  1
 761  5.2495086e+04 5.32e+01 5.73e+02  -2.5 2.24e+01  -0.8 7.09e-03 4.49e-03h  1
 762  5.2490557e+04 4.82e+01 2.71e+03  -2.5 3.04e+01  -1.3 7.92e-03 9.34e-02f  1
 763  5.2465699e+04 4.05e+01 2.94e+03  -2.5 5.39e+01  -1.8 1.91e-02 1.60e-01f  1
 764  5.2461660e+04 4.01e+01 2.95e+03  -2.5 1.36e+02  -2.3 6.71e-02 1.01e-02f  1
 765  5.2412220e+04 3.82e+01 2.94e+03  -2.5 3.84e+02  -2.8 9.72e-02 4.65e-02f  1
 766  5.2404098e+04 3.81e+01 3.07e+03  -2.5 1.12e+03  -3.2 5.07e-02 2.67e-03f  1
 767  5.2362631e+04 3.79e+01 3.11e+03  -2.5 3.27e+03  -3.7 1.79e-02 4.79e-03f  1
 768  5.2299719e+04 3.78e+01 3.26e+03  -2.5 9.80e+03  -4.2 4.83e-02 2.43e-03f  1
 769  5.2129301e+04 3.77e+01 3.30e+03  -2.5 2.87e+04  -4.7 1.53e-02 2.14e-03f  1
iter    objective    inf_pr 

 867  1.3404350e+04 8.32e+01 1.81e+03  -2.5 3.16e+04  -6.3 5.07e-03 5.27e-03f  1
 868  1.3232417e+04 8.34e+01 1.83e+03  -2.5 3.85e+05  -6.7 3.37e-06 4.61e-04f  1
 869  1.2908061e+04 8.42e+01 1.85e+03  -2.5 1.96e+04  -4.5 2.04e-03 4.16e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 870  1.2414069e+04 8.37e+01 1.86e+03  -2.5 5.93e+03  -4.1 1.22e-02 2.35e-02f  1
 871  1.2412903e+04 8.37e+01 1.86e+03  -2.5 1.98e+04  -4.6 1.34e-04 1.42e-05f  1
 872  1.2406776e+04 8.37e+01 2.68e+03  -2.5 7.88e+03  -5.0 1.10e-02 3.81e-04f  1
 873  1.2221875e+04 8.28e+01 1.84e+03  -2.5 9.09e+03  -5.5 3.01e-05 9.42e-03f  1
 874  1.2205065e+04 8.28e+01 1.83e+03  -2.5 2.70e+04  -6.0 4.89e-04 2.49e-04f  1
 875  1.2185704e+04 8.27e+01 1.84e+03  -2.5 2.60e+05  -6.5 1.19e-04 3.97e-04f  1
 876  1.2170664e+04 8.27e+01 2.83e+03  -2.5 2.23e+04  -6.0 1.17e-02 6.31e-04f  1
 877  1.2093636e+04 8.25e+01 2.69e+03  -2.5 5.34e+04  -6.5 3.22e-05 1.64e-03f  1
 878  1.2078017e+04 8.25e+01

 981  1.0088244e+03 2.48e+01 5.66e+03  -2.5 6.91e+01   0.5 3.40e-03 2.80e-03h  1
 982  1.0089153e+03 2.47e+01 2.65e+04  -2.5 3.89e+01   1.8 2.99e-03 1.02e-03h  2
 983  1.0089289e+03 2.47e+01 7.80e+04  -2.5 2.03e+02   2.3 4.29e-04 4.22e-05h  4
 984  1.0089199e+03 2.47e+01 2.48e+04  -2.5 7.66e+01   2.7 6.49e-04 2.52e-04f  1
 985  1.0089170e+03 2.47e+01 2.63e+04  -2.5 3.19e+01   2.2 5.47e-06 3.58e-05f  1
 986  1.0089132e+03 2.47e+01 2.68e+04  -2.5 2.76e+01   1.7 4.69e-06 5.02e-05f  1
 987  1.0089125e+03 2.47e+01 2.52e+04  -2.5 1.91e+01   2.2 7.78e-05 1.33e-05h  1
 988  1.0089041e+03 2.47e+01 2.44e+04  -2.5 2.19e+01   1.7 2.02e-04 1.17e-04f  1
 989  1.0088989e+03 2.47e+01 2.32e+04  -2.5 1.76e+01   2.1 1.78e-04 1.26e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 990  1.0088892e+03 2.47e+01 2.64e+04  -2.5 2.17e+01   1.6 8.28e-05 4.37e-04h  1
 991  1.0088861e+03 2.47e+01 2.28e+04  -2.5 2.45e+01   1.2 1.16e-03 1.28e-04h  1
 992  1.0087831e+03 2.46e+01

1086  4.0618456e+02 3.17e+00 3.77e+03  -2.5 1.60e+01  -1.6 2.15e-02 1.31e-01f  1
1087  4.0611140e+02 3.17e+00 3.90e+03  -2.5 7.43e+00  -1.2 4.30e-01 2.69e-03h  1
1088  4.0384591e+02 3.11e+00 3.69e+03  -2.5 1.34e+01  -1.7 1.04e-03 3.36e-02f  1
1089  4.0268529e+02 3.11e+00 3.64e+03  -2.5 4.17e+03    -  4.63e-03 4.78e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1090  3.9783199e+02 3.11e+00 3.64e+03  -2.5 4.49e+03    -  2.39e-04 1.16e-03f  1
1091  3.9450173e+02 3.11e+00 3.60e+03  -2.5 3.98e+03    -  5.25e-03 1.50e-03f  1
1092  3.9326644e+02 3.01e+00 3.53e+03  -2.5 7.31e+00  -1.2 6.66e-03 3.44e-02f  1
1093  3.9319895e+02 3.01e+00 3.47e+03  -2.5 4.05e+03    -  5.58e-03 2.35e-05f  1
1094  3.7590435e+02 3.17e+00 3.44e+03  -2.5 3.86e+03    -  8.66e-03 8.06e-03f  1
1095  3.0768598e+02 6.56e+00 3.15e+03  -2.5 3.52e+03    -  5.36e-02 3.63e-02f  1
1096  2.3924875e+02 1.09e+01 3.45e+03  -2.5 2.92e+03    -  4.11e-04 5.16e-02f  1
1097  2.2758187e+02 1.10e+01

1219  6.5623410e+01 1.01e+00 1.20e+03  -3.8 2.72e+02    -  6.46e-02 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1220  6.5661592e+01 3.91e-01 2.33e+02  -3.8 1.04e-01  -0.6 7.69e-01 1.00e+00h  1
1221  6.5655107e+01 1.01e+00 1.31e+03  -3.8 1.06e+00  -1.0 1.00e+00 1.05e-01h  2
1222  6.5522193e+01 1.60e+01 1.05e+03  -3.8 7.15e+02    -  1.43e-01 2.24e-01h  1
1223  6.5516790e+01 1.37e+01 6.24e+02  -3.8 1.57e+02    -  3.23e-01 1.42e-01h  1
1224  6.5516745e+01 7.14e+00 1.12e+02  -3.8 2.80e-02  -0.6 1.00e+00 4.76e-01h  1
1225  6.5483893e+01 4.43e+00 2.03e+02  -3.8 2.45e+02    -  1.86e-01 3.70e-01h  1
1226  6.5485204e+01 1.92e+00 6.43e+01  -3.8 1.15e-01  -1.1 1.00e+00 5.53e-01h  1
1227  6.5371433e+01 1.68e+00 8.92e+02  -3.8 4.11e+02    -  1.84e-01 8.75e-01h  1
1228  6.5394634e+01 1.01e-06 1.30e+00  -3.8 2.71e-02  -0.7 1.00e+00 1.00e+00h  1
1229  6.5393875e+01 9.67e-08 3.62e+00  -3.8 9.94e-02  -1.1 1.00e+00 1.00e+00H  1
iter    objective    inf_pr 

In [10]:
solve_pyomo(para_truth[6])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 3.0910763e+04 9.25e+04 1.35e+05  -4.5 7.87e+01  -0.6 3.75e-06 5.07e-01f  1
 188r 3.0934489e+04 9.25e+04 1.33e+05  -4.5 2.26e+02  -1.0 1.44e-02 3.01e-02f  1
 189r 3.1013874e+04 9.24e+04 1.30e+05  -4.5 6.23e+02  -1.5 3.22e-05 4.19e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 3.1013905e+04 9.24e+04 1.14e+05  -4.5 2.53e+02  -1.1 1.30e-01 4.14e-05f  1
 191r 3.1037783e+04 9.24e+04 1.10e+05  -4.5 6.87e+02  -1.6 2.82e-02 1.14e-02f  1
 192r 3.1038029e+04 9.24e+04 1.08e+05  -4.5 2.82e+02  -1.1 1.92e-02 3.27e-04f  1
 193r 3.1039872e+04 9.24e+04 1.06e+05  -4.5 7.54e+02  -1.6 2.55e-02 8.45e-04f  1
 194r 3.1042980e+04 9.24e+04 1.05e+05  -4.5 3.10e+02  -1.2 2.67e-03 2.89e-03f  1
 195r 3.1174786e+04 9.24e+04 1.04e+05  -4.5 8.28e+02  -1.7 5.70e-03 4.38e-02f  1
 196r 3.1174807e+04 9.24e+04 9.59e+04  -4.5 3.43e+02  -1.2 8.26e-02 1.77e-05f  1
 197r 3.1181984e+04 9.24e+04 8.88e+04  -4.5 9.07e+02  -1.7 8.15e-02 2.15e-03f  1
 198r 3.1352336e+04 9.23e+04

 368r 4.0503264e+04 9.00e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 4.34e-12R  2
 369r 4.0320272e+04 9.00e+04 9.99e+02   5.0 2.93e+09    -  5.88e-10 2.45e-08f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 4.0322711e+04 9.00e+04 1.81e+03   2.9 4.99e+05    -  2.16e-03 5.99e-06f  1
 371r 4.1909498e+04 8.96e+04 1.79e+03   2.9 2.66e+05    -  1.28e-03 1.55e-03f  1
 372r 4.2889297e+04 8.94e+04 7.64e+03   2.9 2.74e+05    -  8.46e-03 9.21e-04f  1
 373r 4.4143950e+04 8.90e+04 1.06e+04   2.9 3.29e+05    -  3.60e-03 9.71e-04f  1
 374r 4.4699832e+04 8.89e+04 1.33e+04   2.9 2.45e+05    -  2.12e-03 6.02e-04f  1
 375r 4.6785790e+04 8.83e+04 1.77e+04   2.9 2.21e+05    -  5.01e-03 2.61e-03f  1
 376r 4.7664978e+04 8.81e+04 1.97e+04   2.9 2.02e+05    -  2.30e-03 1.40e-03f  1
 377r 4.8672418e+04 8.77e+04 2.52e+04   2.9 1.79e+05    -  4.72e-03 2.39e-03f  1
 378r 5.0093495e+04 8.71e+04 1.77e+04   2.2 2.06e+05    -  6.30e-04 3.43e-03f  1
 379r 5.0118467e+04 8.71e+04

 663  2.9968025e+05 2.76e+02 1.07e+06  -1.0 1.34e+03  -1.6 3.48e-02 9.76e-03h  1
 664  2.9977165e+05 2.71e+02 1.00e+06  -1.0 1.35e+03  -2.1 1.11e-01 1.70e-02h  1
 665  2.9989360e+05 2.65e+02 9.62e+05  -1.0 1.28e+03  -1.7 7.03e-04 2.27e-02h  1
 666  2.9990003e+05 2.65e+02 8.76e+05  -1.0 1.89e+03  -2.1 1.11e-01 1.23e-03h  1
 667  3.0001677e+05 2.59e+02 8.49e+05  -1.0 1.96e+03  -1.7 7.46e-03 2.22e-02h  1
 668  3.0008714e+05 2.55e+02 6.51e+05  -1.0 1.89e+03  -2.2 2.74e-01 1.41e-02h  1
 669  3.0031218e+05 2.44e+02 5.98e+05  -1.0 1.91e+03  -1.8 4.79e-03 4.53e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 670  3.0031585e+05 2.43e+02 6.40e+05  -1.0 1.77e+03  -0.4 2.66e-03 7.50e-04h  1
 671  3.0031599e+05 2.43e+02 1.07e+06  -1.0 1.82e+03  -0.9 3.64e-02 2.84e-05h  1
 672  3.0032394e+05 2.43e+02 1.07e+06  -1.0 1.86e+03  -1.4 2.76e-03 1.69e-03h  1
 673  3.0036256e+05 2.41e+02 1.79e+06  -1.0 1.80e+03  -1.0 7.80e-02 8.17e-03h  1
 674  3.0069001e+05 2.24e+02

 787  2.7786284e+05 3.35e+01 1.15e+05  -1.0 8.21e+01  -2.4 3.48e-02 7.08e-03h  1
 788  2.7783450e+05 3.35e+01 1.15e+05  -1.0 5.40e+04    -  1.07e-03 3.93e-04f  1
 789  2.7777396e+05 3.35e+01 1.15e+05  -1.0 1.55e+05    -  7.09e-07 2.76e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 790  2.7761514e+05 3.34e+01 1.14e+05  -1.0 4.24e+04    -  1.28e-02 2.62e-03f  1
 791  2.7750026e+05 3.34e+01 1.14e+05  -1.0 4.57e+04    -  3.16e-04 1.60e-03f  1
 792  2.7749815e+05 3.34e+01 1.12e+05  -1.0 3.92e+04    -  1.79e-02 3.67e-05f  1
 793  2.7738745e+05 3.33e+01 1.12e+05  -1.0 3.77e+04    -  1.97e-04 2.06e-03f  1
 794  2.7733254e+05 3.33e+01 1.10e+05  -1.0 3.46e+04    -  1.67e-02 1.20e-03f  1
 795  2.7466903e+05 4.18e+01 1.09e+05  -1.0 3.89e+04    -  2.36e-04 4.50e-02f  1
 796  2.7369901e+05 4.27e+01 1.07e+05  -1.0 5.69e+04    -  2.40e-02 1.05e-02f  1
 797  2.7364668e+05 4.26e+01 9.57e+04  -1.0 4.95e+04    -  8.11e-02 7.99e-04f  1
 798  2.7310459e+05 4.30e+01

...

1336  7.6587443e+04 6.17e+01 9.57e+07  -2.5 3.38e+01   3.4 2.33e-01 1.72e-03h  1
1337  7.6590438e+04 5.97e+01 9.31e+07  -2.5 3.37e+01   2.9 2.39e-02 3.14e-02h  1
1338  7.6591088e+04 5.93e+01 1.04e+08  -2.5 3.27e+01   3.3 4.80e-01 7.03e-03h  1
1339  7.6594540e+04 5.71e+01 1.00e+08  -2.5 3.24e+01   2.9 3.88e-02 3.76e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1340  7.6595688e+04 5.64e+01 9.84e+07  -2.5 3.12e+01   3.3 8.55e-02 1.30e-02h  1
1341  7.6598910e+04 5.43e+01 9.12e+07  -2.5 3.08e+01   2.8 8.12e-02 3.70e-02h  1
1342  7.6598964e+04 5.42e+01 8.99e+07  -2.5 2.97e+01   3.2 8.21e-02 6.35e-04h  1
1343  7.6603532e+04 5.13e+01 7.08e+07  -2.5 2.96e+01   2.8 2.38e-01 5.44e-02h  1
1344  7.6603753e+04 5.11e+01 7.02e+07  -2.5 2.80e+01   3.2 3.59e-02 2.78e-03h  1
1345  7.6608486e+04 4.81e+01 6.40e+07  -2.5 2.79e+01   2.7 9.05e-02 5.98e-02h  1
1346  7.6609134e+04 4.77e+01 5.38e+07  -2.5 2.63e+01   3.1 5.45e-01 8.71e-03h  1
1347  7.6615264e+04 4.37e+01

1448  5.5470704e+04 2.25e+01 2.00e+03  -2.5 1.36e+03  -3.3 6.89e-01 2.22e-01f  1
1449  5.5467572e+04 2.24e+01 2.96e+03  -2.5 5.32e+02  -2.9 1.84e-01 2.16e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1450  5.5467324e+04 2.24e+01 1.47e+03  -2.5 1.41e+03  -3.4 7.03e-01 6.42e-05f  1
1451  5.1168822e+04 2.25e+01 2.37e+03  -2.5 4.02e+03  -3.8 1.50e-03 3.91e-01f  1
1452  5.1004630e+04 2.24e+01 2.33e+03  -2.5 1.63e+04  -4.3 4.26e-02 3.80e-03f  1
1453  5.0965313e+04 2.24e+01 2.22e+03  -2.5 5.24e+04  -4.8 4.96e-03 2.72e-04f  1
1454  5.0777527e+04 2.24e+01 1.82e+03  -2.5 1.49e+04  -4.4 1.50e-01 4.58e-03f  1
1455  4.8653586e+04 2.78e+01 1.56e+03  -2.5 6.31e+04  -4.9 5.10e-03 1.22e-02f  1
1456  4.8644803e+04 2.78e+01 1.74e+03  -2.5 6.03e+06  -5.3 3.62e-05 4.66e-07f  1
1457  4.8464662e+04 2.77e+01 2.37e+03  -2.5 3.23e+04  -4.9 1.53e-02 1.67e-03f  1
1458  4.8284796e+04 2.77e+01 2.31e+03  -2.5 1.99e+05  -5.4 1.32e-05 2.41e-04f  1
1459  4.8213704e+04 2.77e+01

1563  3.1337756e+03 1.13e+02 8.45e+03  -2.5 3.68e+03  -7.3 2.95e-05 4.78e-02f  1
1564  3.0426867e+03 1.10e+02 8.87e+03  -2.5 7.59e+02  -3.2 1.28e-02 2.98e-02f  1
1565  3.0424510e+03 1.10e+02 8.40e+03  -2.5 1.45e+03  -3.7 1.35e-02 4.25e-05f  1
1566  2.9975308e+03 1.09e+02 4.83e+03  -2.5 1.57e+03  -4.2 1.56e-01 1.10e-02f  1
1567  2.9211103e+03 1.07e+02 5.17e+03  -2.5 2.14e+03  -4.6 7.42e-04 1.50e-02f  1
1568  2.8592884e+03 1.06e+02 4.94e+03  -2.5 2.36e+03  -5.1 6.34e-02 1.12e-02f  1
1569  2.6947122e+03 1.03e+02 4.41e+03  -2.5 2.49e+03  -5.6 1.99e-04 2.93e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1570  2.6784222e+03 1.03e+02 4.95e+03  -2.5 2.38e+03  -6.1 5.50e-02 3.17e-03f  1
1571  2.5220318e+03 9.95e+01 5.21e+03  -2.5 2.42e+03  -6.6 6.86e-02 3.05e-02f  1
1572  2.5116575e+03 9.92e+01 5.29e+03  -2.5 7.00e+04    -  1.58e-04 2.22e-03f  1
1573  2.5115364e+03 9.92e+01 5.29e+03  -2.5 2.38e+06    -  1.58e-06 2.41e-06f  1
1574  2.5114543e+03 9.92e+01

1674  4.1118978e+02 1.51e+01 9.85e+03  -2.5 3.80e-01   0.8 5.29e-02 7.58e-01h  1
1675  4.0921568e+02 8.26e-01 2.02e+03  -2.5 4.10e-01   0.3 3.32e-01 1.00e+00h  1
1676  4.0911298e+02 4.58e-01 5.71e+02  -2.5 2.78e-02   1.6 9.86e-01 4.48e-01h  1
1677  4.0867738e+02 5.15e-01 3.51e+01  -2.5 8.53e-02   1.2 9.92e-01 9.49e-01f  1
1678  4.0826868e+02 6.86e-01 2.40e+01  -2.5 2.12e-01   0.7 5.98e-01 3.73e-01f  1
1679  4.0772491e+02 1.01e+00 2.87e+02  -2.5 6.59e-01   0.2 3.56e-01 1.43e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1680  4.0632209e+02 2.49e+00 2.81e+01  -2.5 2.21e-01   0.6 1.00e+00 1.00e+00f  1
1681  4.0612446e+02 2.47e+00 9.96e+02  -2.5 1.45e+00   0.2 1.59e-01 2.93e-02f  1
1682  4.0611065e+02 2.44e+00 8.95e+02  -2.5 2.27e-01   0.6 1.81e-01 1.22e-02h  1
1683  4.0569667e+02 2.41e+00 1.92e+03  -2.5 4.71e-01   0.1 1.00e+00 1.13e-01f  1
1684  4.0242172e+02 1.39e+00 8.52e+02  -2.5 1.30e+00  -0.4 9.97e-01 5.24e-01f  1
1685  4.0032327e+02 1.19e+00

1792  1.1316922e+02 8.62e-02 1.49e+03  -2.5 3.37e-01   1.0 5.15e-01 2.05e-01f  1
1793  1.1239587e+02 1.75e-01 1.45e+03  -2.5 1.55e+03    -  1.02e-01 1.17e-02f  1
1794  1.1241224e+02 1.66e-01 1.11e+03  -2.5 1.11e-01   0.5 1.00e+00 6.25e-02f  5
1795  1.0911740e+02 1.78e+00 9.98e+02  -2.5 8.94e+02    -  1.27e-01 8.87e-02f  1
1796  1.0054181e+02 1.98e+01 1.20e+03  -2.5 6.64e+02    -  9.30e-02 3.71e-01f  1
1797  1.0051982e+02 1.97e+01 9.57e+02  -2.5 8.65e+02    -  5.65e-02 1.22e-03h  1
1798  1.0032265e+02 1.96e+01 9.44e+02  -2.5 1.26e+03    -  2.27e-02 7.63e-03h  1
1799  1.0056773e+02 1.93e+01 1.14e+03  -2.5 5.85e+02    -  1.81e-01 1.93e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1800  9.7047132e+01 1.29e+01 1.93e+03  -2.5 6.15e+02    -  2.38e-03 3.40e-01f  1
1801  9.7377564e+01 2.53e+00 1.91e+03  -2.5 6.26e-02   0.9 1.00e+00 8.08e-01h  1
1802  9.7517708e+01 4.77e-06 2.24e+00  -2.5 3.87e-02   1.4 1.00e+00 1.00e+00h  1
1803  9.7536691e+01 1.36e-05

In [11]:
solve_pyomo(para_truth[7])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 8.6509917e+03 9.75e+04 6.93e+02  -4.5 9.24e+01  -0.4 5.47e-02 7.59e-01f  1
 188r 8.6509972e+03 9.75e+04 5.04e+02  -4.5 4.35e+00   0.9 2.57e-01 4.12e-04f  1
 189r 8.6910887e+03 9.75e+04 4.18e+02  -4.5 1.30e+01   0.4 3.50e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 8.7118092e+03 9.75e+04 5.97e+01  -4.5 3.89e+01  -0.0 5.38e-01 1.74e-01f  1
 191r 8.7124523e+03 9.75e+04 6.10e+01  -4.5 1.46e+01   0.4 4.16e-05 1.44e-02f  1
 192r 8.7130650e+03 9.75e+04 6.12e+01  -4.5 4.37e+01  -0.1 5.18e-05 4.58e-03f  1
 193r 8.7130659e+03 9.75e+04 4.64e+02  -4.5 3.64e+00   1.2 5.53e-02 1.40e-04f  1
 194r 8.7307510e+03 9.75e+04 3.64e+01  -4.5 6.16e+00   0.8 1.00e+00 9.42e-01f  1
 195r 8.7475907e+03 9.74e+04 1.63e+02  -4.5 1.85e+01   0.3 1.00e+00 2.98e-01f  1
 196r 8.8459270e+03 9.74e+04 2.49e+02  -4.5 5.52e+01  -0.2 1.00e+00 5.80e-01f  1
 197r 9.2606197e+03 9.73e+04 9.27e+02  -4.5 1.63e+02  -0.7 1.92e-01 8.23e-01f  1
 198r 9.2607134e+03 9.73e+04

 489  2.1308017e+05 2.78e+03 1.35e+06  -1.0 2.78e+03  -0.9 5.77e-03 1.70e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  2.1308987e+05 2.78e+03 2.03e+06  -1.0 2.78e+03  -0.5 5.02e-02 1.17e-03h  1
 491  2.1309902e+05 2.77e+03 2.05e+06  -1.0 2.78e+03  -1.0 8.40e-03 1.31e-03h  1
 492  2.1311583e+05 2.77e+03 2.14e+06  -1.0 2.77e+03  -0.6 9.65e-03 2.35e-03h  1
 493  2.1313216e+05 2.76e+03 2.14e+06  -1.0 2.77e+03  -1.0 3.42e-03 2.53e-03h  1
 494  2.1313561e+05 2.76e+03 2.25e+06  -1.0 2.76e+03  -0.6 9.48e-03 5.02e-04h  1
 495  2.1314439e+05 2.75e+03 2.25e+06  -1.0 2.76e+03  -1.1 3.28e-03 1.49e-03h  1
 496  2.1314806e+05 2.75e+03 2.68e+06  -1.0 2.75e+03  -0.7 3.79e-02 5.54e-04h  1
 497  2.1316098e+05 2.75e+03 2.69e+06  -1.0 2.75e+03  -1.1 6.73e-03 2.30e-03h  1
 498  2.1316553e+05 2.74e+03 2.79e+06  -1.0 2.75e+03  -0.7 1.02e-02 7.75e-04h  1
 499  2.1317996e+05 2.73e+03 2.78e+06  -1.0 2.74e+03  -1.2 8.08e-04 3.24e-03h  1
iter    objective    inf_pr 

 598  2.1807309e+05 6.30e+02 5.12e+05  -1.0 1.49e+03  -1.4 1.60e-01 5.48e-03h  1
 599  2.1809836e+05 6.24e+02 5.07e+05  -1.0 1.48e+03  -1.8 1.05e-02 9.01e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 600  2.1810246e+05 6.23e+02 5.16e+05  -1.0 1.46e+03  -1.4 1.61e-02 1.39e-03h  1
 601  2.1811820e+05 6.20e+02 5.14e+05  -1.0 1.42e+03  -1.9 1.71e-03 6.10e-03h  1
 602  2.1812129e+05 6.19e+02 5.58e+05  -1.0 1.43e+03  -1.5 7.37e-02 1.09e-03h  1
 603  2.1815188e+05 6.12e+02 5.52e+05  -1.0 1.40e+03  -1.9 8.90e-03 1.18e-02h  1
 604  2.1816301e+05 6.09e+02 6.45e+05  -1.0 1.40e+03  -1.5 1.45e-01 3.97e-03h  1
 605  2.1819101e+05 6.02e+02 6.38e+05  -1.0 1.35e+03  -2.0 5.68e-03 1.16e-02h  1
 606  2.1819884e+05 6.00e+02 6.38e+05  -1.0 1.34e+03  -1.6 5.41e-03 3.14e-03h  1
 607  2.1821373e+05 5.96e+02 5.98e+05  -1.0 1.30e+03  -2.0 1.43e-01 6.74e-03h  1
 608  2.1823542e+05 5.91e+02 5.95e+05  -1.0 1.30e+03  -1.6 1.36e-02 9.15e-03h  1
 609  2.1825677e+05 5.84e+02

 704  2.1933472e+05 8.53e+01 2.37e+06  -1.0 4.35e+02  -1.4 1.19e-03 1.33e-01h  1
 705  2.1933483e+05 8.51e+01 2.25e+06  -1.0 4.44e+02  -1.8 1.60e-02 1.59e-03h  1
 706  2.1933019e+05 8.50e+01 2.19e+06  -1.0 4.05e+03  -2.3 7.04e-03 1.96e-03f  1
 707  2.1933023e+05 8.43e+01 2.23e+06  -1.0 5.99e+02  -1.9 7.19e-04 7.67e-03f  1
 708  2.1933042e+05 8.42e+01 2.62e+06  -1.0 5.51e+02  -0.6 1.99e-02 7.67e-04h  1
 709  2.1933762e+05 8.12e+01 2.53e+06  -1.0 3.34e+02  -1.0 1.12e-03 3.59e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 710  2.1933774e+05 8.12e+01 2.73e+06  -1.0 3.47e+02  -0.6 3.09e-02 5.13e-04h  1
 711  2.1934534e+05 7.84e+01 2.50e+06  -1.0 4.28e+02  -1.1 8.24e-02 3.45e-02h  1
 712  2.1934544e+05 7.83e+01 2.55e+06  -1.0 3.54e+02  -0.7 7.40e-03 4.28e-04h  1
 713  2.1934564e+05 7.83e+01 2.55e+06  -1.0 3.04e+02  -1.1 1.17e-03 7.61e-04h  1
 714  2.1934951e+05 7.69e+01 2.41e+06  -1.0 3.57e+02  -0.7 1.62e-03 1.78e-02h  1
 715  2.1937987e+05 6.28e+01

...

1141  1.4384307e+03 6.97e-01 2.19e+03  -2.5 6.45e+01  -2.3 1.54e-01 1.75e-01f  1
1142  1.4289019e+03 7.05e-01 2.23e+03  -2.5 2.56e+02  -2.8 4.78e-03 9.61e-03f  1
1143  1.4276883e+03 6.99e-01 2.26e+03  -2.5 4.95e+01  -1.5 1.84e-02 8.80e-03f  1
1144  1.4262924e+03 6.92e-01 2.41e+03  -2.5 3.17e+01  -2.0 9.65e-02 1.00e-02f  1
1145  1.4189337e+03 6.80e-01 2.43e+03  -2.5 7.71e+01  -2.4 3.38e-02 2.28e-02f  1
1146  1.3842523e+03 8.75e-01 2.37e+03  -2.5 2.56e+02  -2.9 9.11e-03 3.31e-02f  1
1147  1.3838380e+03 8.73e-01 2.73e+03  -2.5 5.89e+01  -1.6 5.52e-02 2.56e-03f  1
1148  1.3823506e+03 8.64e-01 1.22e+04  -2.5 3.28e+01  -2.1 1.00e+00 1.06e-02f  1
1149  1.3567922e+03 9.40e-01 1.23e+04  -2.5 9.15e+01  -2.5 2.00e-01 6.81e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1150  1.1529611e+03 1.06e+01 1.03e+04  -2.5 3.30e+02  -3.0 3.56e-02 1.53e-01f  1
1151  1.0972204e+03 1.09e+01 9.78e+03  -2.5 1.36e+02  -2.6 3.47e-02 9.58e-02f  1
1152  1.0960510e+03 1.09e+01

1254  4.3079316e+02 1.46e-01 6.35e+03  -3.8 1.56e+03  -4.4 1.02e-03 4.88e-04f  1
1255  4.2193564e+02 1.63e-01 6.17e+03  -3.8 8.36e+02  -4.9 6.43e-03 1.41e-02f  1
1256  4.1710934e+02 1.68e-01 6.13e+03  -3.8 9.48e+02  -5.4 7.19e-03 6.89e-03f  1
1257  4.1710936e+02 1.68e-01 4.10e+03  -3.8 1.92e-02   2.5 2.68e-02 1.24e-03h  1
1258  4.1710986e+02 1.68e-01 2.00e+04  -3.8 3.43e-02   2.0 3.89e-02 2.09e-03f  2
1259  4.1713840e+02 1.58e-01 9.35e+03  -3.8 7.28e-02   1.5 2.99e-02 5.56e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1260  4.1716278e+02 1.52e-01 1.84e+04  -3.8 9.23e-02   1.1 1.11e-01 4.15e-02h  1
1261  4.1721293e+02 1.39e-01 2.34e+04  -3.8 8.61e-02   0.6 3.81e-01 8.69e-02h  3
1262  4.1737257e+02 9.32e-02 1.10e+02  -3.8 9.08e-02   0.1 1.00e+00 3.28e-01h  1
1263  4.1744468e+02 6.23e-02 5.65e+03  -3.8 1.76e-01  -0.4 7.11e-01 3.31e-01h  1
1264  4.1739603e+02 4.42e-02 9.78e+03  -3.8 4.52e-01  -0.9 1.00e+00 2.90e-01h  1
1265  4.1681957e+02 1.68e-02

1379  1.1436105e+02 2.38e+00 4.52e+03  -3.8 8.35e-01  -0.8 2.27e-01 5.84e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1380  1.1439485e+02 1.42e+00 3.53e+03  -3.8 1.20e+00  -1.3 2.51e-01 3.98e-01h  1
1381  1.1438462e+02 1.40e+00 2.65e+03  -3.8 3.05e+00  -1.8 2.26e-01 1.38e-02h  1
1382  1.1436414e+02 1.09e+00 2.20e+03  -3.8 1.34e+00  -1.4 1.74e-01 2.20e-01h  1
1383  1.1430950e+02 1.01e+00 1.25e+03  -3.8 3.12e+00  -1.8 4.28e-01 7.31e-02h  1
1384  1.0984996e+02 1.26e+00 2.42e+03  -3.8 5.94e+02    -  1.91e-02 6.27e-02f  1
1385  1.0764094e+02 1.32e+00 2.05e+03  -3.8 6.07e+02    -  4.30e-02 3.33e-02f  1
1386  1.0755699e+02 1.26e+00 1.81e+03  -3.8 1.57e+01  -2.3 1.60e-01 4.99e-02h  1
1387  1.0755536e+02 1.25e+00 1.44e+03  -3.8 3.31e+00  -1.9 2.56e-01 3.93e-03h  1
1388  1.0675836e+02 1.25e+00 1.20e+03  -3.8 5.73e+02    -  2.17e-02 1.29e-02f  1
1389  9.5070891e+01 4.45e+00 3.54e+03  -3.8 6.36e+02    -  8.71e-02 1.80e-01f  1
iter    objective    inf_pr 

1489  6.6789696e+01 2.53e-01 1.68e+02  -5.7 2.36e-02   0.6 1.00e+00 8.35e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1490  6.6793844e+01 5.58e-04 5.57e-03  -5.7 4.20e-03   0.1 1.00e+00 1.00e+00h  1
1491  6.6793658e+01 3.12e-11 1.25e-03  -5.7 2.89e-03  -0.4 1.00e+00 1.00e+00h  1
1492  6.6793606e+01 2.64e-11 1.14e-03  -5.7 7.87e-03  -0.8 1.00e+00 1.00e+00h  1
1493  6.6793450e+01 3.82e-11 1.12e-03  -5.7 2.33e-02  -1.3 1.00e+00 1.00e+00h  1
1494  6.6792982e+01 9.73e-11 1.12e-03  -5.7 6.96e-02  -1.8 1.00e+00 1.00e+00h  1
1495  6.6792689e+01 1.18e-09 3.05e-01  -5.7 2.04e-01  -2.3 1.00e+00 2.13e-01h  1
1496  6.6792131e+01 4.85e-09 6.32e-01  -5.7 4.67e-01  -2.7 1.00e+00 1.82e-01f  1
1497  6.6791599e+01 1.48e-08 1.62e+00  -5.7 1.03e+00  -3.2 1.00e+00 7.80e-02f  1
1498  6.6790553e+01 3.82e-08 3.00e+00  -5.7 1.91e+00  -3.7 1.00e+00 8.20e-02f  1
1499  6.6788808e+01 4.45e-08 4.10e+00  -5.7 2.75e+00  -4.2 1.00e+00 9.67e-02f  1
iter    objective    inf_pr 

1613  6.3720219e+01 1.63e+01 4.65e+02  -5.7 5.05e+04    -  2.60e-08 2.92e-04f  1
1614  6.3720321e+01 1.35e+01 3.77e+02  -5.7 2.71e-01  -2.7 2.05e-01 1.71e-01h  1
1615  6.3720321e+01 1.35e+01 3.77e+02  -5.7 1.33e+00  -3.2 7.22e-04 8.25e-04h  1
1616  6.3720321e+01 1.35e+01 2.92e+02  -5.7 1.52e+00  -1.9 3.93e-01 2.78e-04h  1
1617  6.3720232e+01 1.35e+01 2.94e+02  -5.7 1.17e+03    -  4.78e-02 1.52e-03h  1
1618  6.3720542e+01 1.35e+01 5.23e+02  -5.7 1.78e+03    -  4.53e-02 3.24e-03h  1
1619  6.3719939e+01 1.22e+01 4.22e+02  -5.7 1.65e+02    -  1.46e-01 9.56e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1620  6.3719948e+01 1.20e+01 3.38e+02  -5.7 2.65e-01  -2.4 2.75e-01 1.44e-02h  1
1621  6.3720073e+01 1.20e+01 7.17e+02  -5.7 1.40e+04    -  5.21e-03 1.71e-04h  5
1622  6.3719797e+01 1.12e+01 5.92e+02  -5.7 1.23e+02    -  1.44e-01 6.78e-02h  1
1623  6.3719816e+01 1.08e+01 3.68e+02  -5.7 1.50e-01  -2.8 4.86e-01 3.80e-02h  1
1624  6.3719822e+01 1.06e+01

In [12]:
solve_pyomo(para_truth[8])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 9.9445095e+03 9.75e+04 6.28e+01  -3.0 3.73e+00   0.0 1.00e+00 3.48e-01f  1
 188r 1.0061735e+04 9.75e+04 5.92e+01  -3.0 3.43e+02    -  5.53e-02 6.70e-02f  1
 189r 1.0171325e+04 9.75e+04 1.86e+03  -3.0 3.20e+02    -  2.86e-01 6.71e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.0259118e+04 9.75e+04 1.40e+03  -3.0 2.99e+02    -  2.19e-03 5.79e-02f  1
 191r 1.0590880e+04 9.74e+04 1.79e+02  -3.0 2.81e+02    -  9.10e-02 2.33e-01f  1
 192r 1.0681257e+04 9.74e+04 1.07e+03  -3.0 2.16e+02    -  2.37e-01 8.25e-02f  1
 193r 1.0749658e+04 9.74e+04 1.20e+03  -3.0 1.98e+02    -  1.12e-01 6.80e-02f  1
 194r 1.0751769e+04 9.74e+04 2.42e+03  -3.0 1.85e+02    -  3.11e-01 2.25e-03f  1
 195r 1.1013821e+04 9.73e+04 1.40e+03  -3.0 1.84e+02    -  1.58e-01 2.81e-01f  1
 196r 1.1018585e+04 9.73e+04 8.82e+01  -3.0 4.18e+00  -0.5 1.00e+00 2.42e-01f  1
 197r 1.1058885e+04 9.73e+04 7.30e+02  -3.0 1.32e+02    -  2.38e-01 6.04e-02f  1
 198r 1.1065020e+04 9.73e+04

 463r 3.3926133e+04 8.99e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 1.12e-08R  2
 464r 3.3844283e+04 8.99e+04 9.99e+02   5.0 5.68e+08    -  5.80e-09 6.12e-08f  1
 465r 3.3841190e+04 8.99e+04 9.02e+03   2.9 3.75e+05    -  6.60e-03 6.43e-05f  1
 466r 3.4927397e+04 8.96e+04 9.06e+03   2.9 2.02e+05    -  2.79e-03 1.55e-03f  1
 467r 3.5383160e+04 8.95e+04 1.13e+04   2.9 2.15e+05    -  3.85e-03 6.21e-04f  1
 468r 3.6345586e+04 8.91e+04 1.65e+04   2.9 2.10e+05    -  6.21e-03 1.69e-03f  1
 469r 3.9163229e+04 8.81e+04 1.69e+04   2.9 2.17e+05    -  6.04e-03 5.09e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470r 4.1575515e+04 8.71e+04 2.61e+04   2.9 3.13e+05    -  8.84e-03 3.56e-03f  1
 471r 4.4490733e+04 8.56e+04 2.00e+04   2.9 6.10e+05    -  4.62e-04 2.76e-03f  1
 472r 4.5999192e+04 8.48e+04 2.46e+04   2.9 8.19e+05    -  2.70e-03 1.05e-03f  1
 473r 4.6604042e+04 8.46e+04 3.75e+04   2.2 7.49e+03  -2.0 6.44e-02 2.95e-02f  1
 474r 4.7576308e+04 8.42e+04

 594  1.4459828e+05 4.00e+03 3.71e+06  -1.0 4.01e+03  -0.7 2.43e-02 2.47e-03h  1
 595  1.4462596e+05 4.00e+03 4.02e+06  -1.0 9.65e+03  -1.2 2.08e-02 2.18e-03h  1
 596  1.4477709e+05 3.95e+03 8.81e+06  -1.0 4.79e+03  -0.8 1.00e-01 1.20e-02h  1
 597  1.4481785e+05 3.94e+03 9.87e+06  -1.0 5.22e+03  -0.4 1.16e-02 3.22e-03h  1
 598  1.4483130e+05 3.93e+03 1.38e+07  -1.0 6.36e+03  -0.8 1.11e-01 1.08e-03h  1
 599  1.4485552e+05 3.92e+03 1.38e+07  -1.0 3.57e+04  -1.3 3.18e-03 2.25e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 600  1.4501805e+05 3.87e+03 1.54e+07  -1.0 4.32e+03  -0.9 5.93e-02 1.36e-02h  1
 601  1.4502621e+05 3.87e+03 1.54e+07  -1.0 6.32e+03  -1.4 3.70e-02 6.87e-04h  1
 602  1.4507488e+05 3.85e+03 1.88e+07  -1.0 4.25e+03  -0.9 1.09e-01 4.19e-03h  1
 603  1.4513749e+05 3.83e+03 1.87e+07  -1.0 7.19e+03  -1.4 2.17e-02 5.36e-03h  1
 604  1.4513939e+05 3.83e+03 2.19e+07  -1.0 4.59e+03  -1.0 1.20e-01 1.64e-04h  1
 605  1.4533879e+05 3.76e+03

 706  1.5337001e+05 8.37e+02 5.84e+07  -1.0 1.30e+03   0.0 1.13e-02 6.88e-04h  1
 707  1.5338799e+05 8.30e+02 5.78e+07  -1.0 1.33e+03  -0.5 2.57e-03 7.98e-03h  1
 708  1.5338812e+05 8.30e+02 5.74e+07  -1.0 1.35e+03  -0.9 8.53e-03 6.03e-05h  1
 709  1.5340457e+05 8.25e+02 5.50e+07  -1.0 1.36e+03  -1.4 3.84e-02 6.56e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 710  1.5344285e+05 8.11e+02 5.47e+07  -1.0 1.34e+03  -1.0 5.57e-04 1.60e-02h  1
 711r 1.5344285e+05 8.11e+02 1.00e+03   2.9 0.00e+00   0.3 0.00e+00 3.59e-07R  5
 712r 1.5105502e+05 6.57e+02 3.00e+04   2.9 3.52e+06    -  4.86e-04 1.89e-04f  1
 713  1.5100973e+05 6.57e+02 5.77e+03  -1.0 1.85e+04    -  2.89e-04 8.94e-04f  1
 714  1.5101207e+05 6.57e+02 5.77e+03  -1.0 6.57e+02  -0.1 1.54e-04 2.21e-04h  1
 715  1.5101302e+05 6.57e+02 5.77e+03  -1.0 6.57e+02  -0.6 1.14e-04 9.44e-05h  1
 716  1.5101511e+05 6.56e+02 5.77e+03  -1.0 6.57e+02  -1.1 2.23e-04 2.66e-04h  1
 717  1.5101746e+05 6.56e+02

 837  9.6069487e+04 7.82e+01 4.72e+03  -1.0 1.15e+02  -0.5 1.12e-02 2.21e-03h  1
 838  9.6074182e+04 7.79e+01 4.81e+03  -1.0 1.32e+02  -0.9 9.04e-03 3.96e-03h  1
 839  9.6078604e+04 7.75e+01 5.47e+03  -1.0 1.77e+02  -1.4 1.35e-02 4.34e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 840  9.6079929e+04 7.75e+01 5.47e+03  -1.0 2.21e+03  -1.9 6.28e-04 6.25e-04h  1
 841  9.6083837e+04 7.70e+01 5.69e+03  -1.0 1.56e+02  -1.5 6.86e-03 5.61e-03h  1
 842  9.6085686e+04 7.69e+01 1.43e+04  -1.0 1.48e+02  -1.0 1.51e-02 2.30e-03h  1
 843  9.6089230e+04 7.65e+01 2.44e+04  -1.0 2.59e+02  -1.5 2.01e-02 4.45e-03h  1
 844  9.6090480e+04 7.64e+01 4.44e+04  -1.0 2.41e+02  -1.1 1.07e-02 1.47e-03h  1
 845  9.6093468e+04 7.61e+01 5.22e+04  -1.0 3.84e+02  -1.6 7.27e-03 3.63e-03h  1
 846  9.6094822e+04 7.60e+01 2.19e+05  -1.0 3.87e+02  -1.1 2.93e-02 1.73e-03h  1
 847  9.6096905e+04 7.58e+01 2.11e+05  -1.0 4.66e+02  -1.6 6.75e-04 3.30e-03h  1
 848  9.6100931e+04 7.55e+01

 975  1.0624834e+02 1.34e-03 9.09e+04  -2.5 9.27e-02   1.1 9.48e-01 1.00e+00h  1
 976  1.0416244e+02 1.09e-01 7.81e+00  -2.5 2.73e-01   0.6 1.00e+00 5.06e-01f  1
 977  1.0149611e+02 2.21e-01 1.30e+01  -2.5 8.38e-01   0.1 1.95e-01 3.06e-01f  1
 978  1.0060523e+02 5.37e-02 4.61e+01  -2.5 3.03e-01   0.6 1.00e+00 2.93e-01f  1
 979  9.8887842e+01 5.13e-01 7.26e+01  -2.5 7.39e-01   0.1 4.52e-01 2.67e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 980  9.7896969e+01 3.47e-01 1.09e+02  -2.5 2.65e-01   0.5 1.00e+00 4.33e-01f  1
 981  9.6798629e+01 6.74e-01 3.39e+02  -2.5 6.42e-01   0.0 7.98e-01 2.15e-01f  1
 982  9.5664557e+01 5.42e-01 1.64e+02  -2.5 2.32e-01   0.5 1.00e+00 6.02e-01f  1
 983  9.4705376e+01 1.41e+00 7.77e+02  -2.5 5.47e-01  -0.0 9.97e-01 2.34e-01f  1
 984  9.4088852e+01 8.72e-01 5.28e+02  -2.5 1.93e-01   0.4 1.00e+00 4.29e-01h  1
 985  9.3829319e+01 7.93e+00 4.88e+02  -2.5 7.02e+00  -0.1 6.72e-02 3.72e-02f  1
 986  9.3764093e+01 7.60e+00

1095  7.1320781e+01 6.74e+00 1.92e+02  -3.8 1.67e-01  -1.4 1.00e+00 1.61e-01h  1
1096  7.1306575e+01 4.76e+00 1.05e+02  -3.8 5.07e+02    -  2.52e-01 2.89e-01h  1
1097  7.1307364e+01 7.60e-03 1.21e+00  -3.8 2.04e-01  -1.0 1.00e+00 1.00e+00h  1
1098  7.1274721e+01 8.00e-02 4.34e+02  -3.8 4.20e+02    -  1.69e-01 7.65e-01h  1
1099  7.1269635e+01 5.49e-02 5.10e+02  -3.8 2.68e+02    -  1.66e-01 7.95e-01H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1100  7.1266727e+01 8.19e-03 6.83e+01  -3.8 6.37e+02    -  6.14e-01 2.23e-01h  1
1101  7.1270133e+01 1.07e-07 3.76e-01  -3.8 1.85e-01  -1.5 1.00e+00 1.00e+00H  1
1102  7.1269652e+01 1.66e+00 7.90e-01  -3.8 1.95e-01  -2.0 1.00e+00 1.00e+00h  1
1103  7.1268734e+01 2.96e+00 3.19e+01  -3.8 3.76e-01  -2.4 1.00e+00 4.46e-01h  2
1104  7.1268447e+01 3.05e-07 7.29e-02  -3.8 2.90e-02  -1.1 1.00e+00 1.00e+00h  1
1105  7.1267816e+01 7.54e-01 5.69e-01  -3.8 1.13e-01  -1.6 1.00e+00 1.00e+00h  1
1106  7.1267821e+01 1.75e-08

In [13]:
solve_pyomo(para_truth[9])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 3.1397654e+04 9.23e+04 2.16e+04  -4.5 1.03e+02  -0.7 6.93e-02 1.66e-01f  1
 188r 3.1397688e+04 9.23e+04 2.10e+04  -4.5 2.96e+02  -1.2 1.59e-02 3.39e-05f  1
 189r 3.1450652e+04 9.23e+04 1.60e+04  -4.5 1.15e+02  -0.7 2.17e-01 1.38e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 3.1477106e+04 9.23e+04 1.56e+04  -4.5 3.30e+02  -1.2 3.01e-02 2.34e-02f  1
 191r 3.1587899e+04 9.23e+04 1.25e+04  -4.5 1.28e+02  -0.8 2.11e-01 2.57e-01f  1
 192r 3.1587902e+04 9.23e+04 1.13e+04  -4.5 6.10e+00   0.5 8.72e-02 1.53e-04f  1
 193r 3.1611702e+04 9.23e+04 1.13e+04  -4.5 1.83e+01   0.1 8.91e-03 3.82e-01f  1
 194r 3.1612091e+04 9.23e+04 4.87e+03  -4.5 6.85e+00   0.5 5.62e-01 1.68e-02f  1
 195r 3.1642720e+04 9.23e+04 4.89e+03  -4.5 2.06e+01   0.0 1.75e-03 4.40e-01f  1
 196r 3.1642767e+04 9.23e+04 6.35e+03  -4.5 1.07e+00   1.3 1.59e-01 1.94e-02f  1
 197r 3.1642809e+04 9.23e+04 1.07e+04  -4.5 3.07e+00   0.9 2.76e-01 4.90e-03f  1
 198r 3.1651774e+04 9.22e+04

 479  2.5790677e+05 2.88e+03 1.67e+06  -1.0 3.63e+03  -0.8 2.16e-02 1.92e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480  2.5791924e+05 2.88e+03 1.93e+06  -1.0 3.62e+03  -0.4 1.33e-02 1.57e-03h  1
 481  2.5793341e+05 2.87e+03 1.98e+06  -1.0 3.61e+03  -0.9 1.01e-02 1.97e-03h  1
 482  2.5794304e+05 2.87e+03 2.01e+06  -1.0 3.60e+03  -0.4 2.81e-03 1.24e-03h  1
 483  2.5795122e+05 2.86e+03 2.29e+06  -1.0 3.62e+03  -0.9 4.73e-02 1.14e-03h  1
 484  2.5797065e+05 2.86e+03 2.38e+06  -1.0 3.57e+03  -0.5 7.39e-03 2.66e-03h  1
 485  2.5798396e+05 2.85e+03 2.38e+06  -1.0 3.59e+03  -1.0 2.30e-03 1.94e-03h  1
 486  2.5798638e+05 2.85e+03 2.64e+06  -1.0 3.54e+03  -0.6 1.45e-02 3.37e-04h  1
 487  2.5800605e+05 2.84e+03 2.65e+06  -1.0 3.56e+03  -1.0 7.54e-03 3.00e-03h  1
 488  2.5801410e+05 2.84e+03 2.69e+06  -1.0 3.54e+03  -0.6 3.93e-03 1.16e-03h  1
 489  2.5803095e+05 2.83e+03 2.87e+06  -1.0 3.52e+03  -1.1 4.91e-02 2.75e-03h  1
iter    objective    inf_pr 

 588  2.6202965e+05 1.12e+03 9.67e+07  -1.0 3.46e+03  -1.4 8.88e-04 1.46e-01h  1
 589  2.6202974e+05 1.12e+03 1.09e+08  -1.0 1.36e+03   1.8 1.78e-03 2.04e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 590  2.6202979e+05 1.12e+03 1.09e+08  -1.0 1.52e+03   1.3 2.74e-05 1.40e-05h  1
 591  2.6204207e+05 1.11e+03 9.68e+07  -1.0 1.90e+03   0.8 1.36e-06 3.78e-03h  1
 592  2.6205333e+05 1.11e+03 9.33e+07  -1.0 2.43e+03   0.4 1.44e-03 3.50e-03h  1
 593  2.6208131e+05 1.10e+03 8.81e+07  -1.0 2.76e+03  -0.1 5.92e-06 8.79e-03h  1
 594  2.6208201e+05 1.10e+03 9.37e+07  -1.0 1.34e+03   2.1 7.14e-04 1.64e-04h  1
 595r 2.6208201e+05 1.10e+03 1.00e+03   3.0 0.00e+00   1.6 0.00e+00 2.91e-07R  3
 596r 2.6220503e+05 1.05e+03 1.66e+04   3.0 1.57e+06    -  3.33e-03 4.18e-05f  1
 597r 2.6297754e+05 6.69e+02 2.33e+04   2.3 3.28e+05    -  6.95e-03 1.57e-03f  1
 598  2.6298030e+05 6.68e+02 5.30e+03  -1.0 6.69e+02   1.2 2.92e-05 1.40e-04h  1
 599  2.6298240e+05 6.68e+02

 697  2.6534285e+05 2.36e+01 1.22e+06  -1.0 6.65e+02  -2.6 2.69e-02 2.32e-02f  1
 698  2.6531907e+05 1.33e+01 7.76e+05  -1.0 6.44e+02  -2.2 3.53e-01 4.30e-01f  1
 699  2.6531741e+05 1.32e+01 1.04e+05  -1.0 3.73e+02  -2.7 3.60e-01 8.00e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 700  2.6531759e+05 1.32e+01 9.50e+04  -1.0 3.86e+02  -2.3 3.46e-02 1.03e-03h  1
 701  2.6531049e+05 9.62e+00 3.67e+05  -1.0 3.82e+02  -2.7 1.27e-03 2.72e-01f  1
 702  2.6528126e+05 5.95e+00 3.31e+04  -1.0 2.74e+02  -2.3 6.05e-01 3.79e-01f  1
 703  2.6527920e+05 5.91e+00 3.88e+04  -1.0 1.74e+02  -2.8 2.16e-02 7.35e-03f  1
 704  2.6526101e+05 5.82e+00 2.65e+04  -1.0 6.43e+02  -3.3 1.86e-04 2.36e-02f  1
 705  2.6526126e+05 5.81e+00 2.66e+04  -1.0 1.43e+03  -2.8 1.19e-03 9.64e-04h  1
 706  2.6526088e+05 5.79e+00 2.69e+04  -1.0 1.96e+02  -2.4 5.22e-03 3.96e-03h  1
 707  2.6526413e+05 5.77e+00 2.51e+04  -1.0 3.15e+03  -2.9 5.00e-05 3.02e-03h  1
 708  2.6526408e+05 5.77e+00

...

1553  5.3055797e+04 2.34e+01 8.98e+03  -3.8 1.95e+04  -4.6 2.32e-06 5.82e-03f  1
1554  5.3052486e+04 2.34e+01 8.38e+03  -3.8 1.09e+05  -5.1 5.70e-03 1.15e-05f  1
1555  4.5343106e+04 5.17e+01 1.07e+04  -3.8 2.60e+04  -4.6 4.26e-06 1.09e-01f  1
1556  4.4586847e+04 5.08e+01 9.64e+03  -3.8 8.99e+03  -4.2 1.76e-01 3.00e-02f  1
1557  4.2754316e+04 4.45e+01 8.55e+03  -3.8 3.28e+03  -3.8 5.35e-03 2.00e-01f  1
1558  4.2631117e+04 4.30e+01 6.68e+03  -3.8 1.25e+03  -3.4 6.90e-01 3.50e-02f  1
1559  3.9207483e+04 4.42e+01 3.00e+03  -3.8 3.74e+03  -3.8 5.85e-05 3.30e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1560  3.9207071e+04 4.42e+01 2.03e+03  -3.8 1.44e+03  -3.4 5.37e-01 1.02e-04f  1
1561  3.9181492e+04 4.41e+01 1.78e+03  -3.8 3.58e+04  -3.9 2.61e-05 8.91e-04f  1
1562  3.8092612e+04 3.30e+01 3.27e+03  -3.8 1.55e+03  -3.5 7.37e-02 2.49e-01f  1
1563  3.7930077e+04 2.99e+01 5.71e+02  -3.8 5.88e+02  -3.0 3.62e-01 9.56e-02f  1
1564  3.7922284e+04 2.98e+01

1669  6.6039541e+03 4.15e+01 1.29e+05  -3.8 7.03e+03    -  3.05e-03 6.43e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1670  6.6038732e+03 4.15e+01 1.30e+05  -3.8 2.21e+02  -2.6 1.28e-02 2.02e-04f  1
1671  6.3525771e+03 4.09e+01 1.29e+05  -3.8 6.94e+03    -  1.68e-04 1.74e-02f  1
1672  6.3525669e+03 4.09e+01 1.61e+05  -3.8 1.10e+02  -2.2 3.86e-01 4.32e-04h  1
1673  5.9159902e+03 4.10e+01 1.61e+05  -3.8 2.92e+04    -  7.70e-04 8.25e-03f  1
1674  5.9159825e+03 4.10e+01 1.60e+05  -3.8 2.22e+02  -2.7 1.62e-02 1.61e-05h  1
1675  5.9159381e+03 4.10e+01 1.61e+05  -3.8 7.60e+05    -  3.15e-04 3.89e-08f  1
1676  5.2964802e+03 4.33e+01 1.60e+05  -3.8 3.53e+04    -  2.00e-07 1.07e-02f  1
1677  5.2921142e+03 4.11e+01 1.53e+05  -3.8 1.08e+02  -2.3 1.17e-02 5.08e-02f  1
1678  5.2576779e+03 3.90e+01 1.31e+05  -3.8 2.31e+02  -2.7 2.55e-01 5.80e-02f  1
1679  5.0279125e+03 3.60e+01 1.20e+05  -3.8 5.85e+02  -3.2 4.60e-02 1.20e-01f  1
iter    objective    inf_pr 

1816  1.6181770e+02 1.13e+01 3.57e+03  -3.8 5.79e+02    -  1.73e-02 4.38e-02f  1
1817  1.6081000e+02 1.11e+01 3.46e+03  -3.8 5.66e+02    -  3.46e-02 1.83e-02f  1
1818  1.6077722e+02 1.09e+01 3.04e+03  -3.8 7.17e-01   0.1 8.24e-02 1.58e-02h  1
1819  1.6074819e+02 1.09e+01 3.03e+03  -3.8 5.94e+02    -  3.83e-03 5.58e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1820  1.6061876e+02 1.09e+01 3.04e+03  -3.8 2.26e+03    -  7.40e-08 2.17e-04f  1
1821  1.6041525e+02 1.03e+01 2.75e+03  -3.8 9.16e-01  -0.4 9.05e-02 5.14e-02f  1
1822  1.6011507e+02 1.00e+01 2.67e+03  -3.8 1.98e+00  -0.8 2.98e-02 3.27e-02h  1
1823  1.5953503e+02 9.87e+00 2.70e+03  -3.8 6.83e+00  -1.3 6.14e-04 1.75e-02f  1
1824  1.5925295e+02 9.88e+00 2.63e+03  -3.8 1.71e+02  -1.8 1.70e-03 4.49e-04f  1
1825  1.5616280e+02 1.02e+01 2.74e+03  -3.8 7.80e+00  -1.4 9.83e-03 9.60e-02f  1
1826  1.5564121e+02 1.02e+01 2.54e+03  -3.8 1.83e+01  -1.9 3.29e-02 5.29e-03f  1
1827  1.5548682e+02 1.02e+01

1933  8.3060020e+01 1.22e-01 7.63e+00  -3.8 7.00e-02   0.9 1.00e+00 1.00e+00h  1
1934  8.3044240e+01 1.40e-01 1.40e+03  -3.8 2.59e-01   0.4 7.79e-01 1.20e-01h  1
1935  8.3038064e+01 1.05e-01 1.03e+03  -3.8 4.65e-02   0.9 1.00e+00 2.85e-01h  1
1936  8.3023877e+01 1.28e-01 1.27e+03  -3.8 3.64e-01   0.4 1.00e+00 1.07e-01h  1
1937  8.3011896e+01 3.85e-02 9.86e-01  -3.8 2.95e-02   0.8 1.00e+00 1.00e+00h  1
1938  8.2979330e+01 4.19e-01 1.77e+02  -3.8 2.36e-01   0.3 1.00e+00 4.18e-01h  1
1939  8.2329376e+01 1.52e+00 8.11e+02  -3.8 6.82e+02    -  3.22e-01 9.12e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1940  8.0614821e+01 1.25e+01 2.58e+02  -3.8 6.49e+02    -  1.24e-01 2.81e-01f  1
1941  8.0680467e+01 8.77e+00 1.79e+03  -3.8 1.24e-01   0.8 1.00e+00 3.16e-01h  1
1942  8.0485864e+01 7.99e+00 1.06e+03  -3.8 3.12e+02    -  3.50e-01 1.05e-01h  1
1943  8.0469317e+01 7.49e+00 9.34e+02  -3.8 1.04e+03    -  9.54e-02 6.42e-02h  1
1944  8.0412104e+01 5.92e+00

2053  7.8662394e+01 3.39e-09 3.44e-05  -5.7 3.58e-02  -3.0 1.00e+00 1.00e+00h  1
2054  7.8662225e+01 1.79e-08 2.95e-05  -5.7 9.21e-02  -3.5 1.00e+00 1.00e+00h  1
2055  7.8661856e+01 9.90e-08 4.36e-02  -5.7 2.26e-01  -4.0 1.00e+00 9.31e-01h  1
2056  7.8660766e+01 1.32e-06 5.82e-02  -5.7 6.51e-01  -4.4 9.67e-01 1.00e+00f  1
2057  7.8657481e+01 1.41e-05 9.33e-04  -5.7 1.88e+00  -4.9 1.00e+00 1.00e+00h  1
2058  7.8647637e+01 4.88e-04 9.92e+00  -5.7 5.64e+00  -5.4 3.88e-01 1.00e+00h  1
2059  7.8617908e+01 1.37e-02 1.95e+01  -5.7 1.70e+01  -5.9 6.68e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2060  7.8529512e+01 2.98e-01 9.87e+01  -5.7 5.05e+01  -6.4 4.04e-01 1.00e+00h  1
2061  7.8378167e+01 2.04e+00 6.22e+01  -5.7 1.30e+02  -6.8 6.22e-01 6.87e-01h  1
2062  7.8330113e+01 2.05e+00 2.28e+02  -5.7 2.38e+02  -7.3 5.84e-01 1.39e-01h  1
2063  7.8244445e+01 2.84e+00 5.50e+02  -5.7 3.83e+02  -7.8 7.10e-01 1.60e-01h  1
2064  7.8244521e+01 2.83e+00